In [1]:
# This chapter is all about Maps, Hash Tables and Skip Lists

# Recap

# Dicts are associative arrays or maps.
# They are implemented as hash tables which has a hash code and compression fucntion. 
# 
# hash code takes a string and returns an index between -inf and +inf
# compression fucntion makes that index in between 0 and N - 1 
# where N is the bucket array size.

"""Cool info"""

# Python dicts actually store the hash value for their keys, beacause computing 
# hash methods each time is costly

# Collison handling schemes

# SeperatE Chaining or Open Adressingp

# seperate chainign is like holding long lists in each slot of the bucket array to avoid coillisons

# Open addressing is things like, linear probing, quadratic probing, doublke hsahing.
# linear probing - if occupied go next
# quadratic probing - if occupied go next but i ^2
# double hashing - insert another hash to the calculation

"""Cool info"""

# Python dicts use Open Addressing with a pseudo random number generator.
# With load factor threshold 2/3

# load factor should NOT go over 1. For python dicts, if it goes over 2/3, 
# bucket array is resized.

# IN default python dictionaries will initialize woth bucket array with size 8

"""Sorted Maps"""

# Keys are sorted.
# these are really good for finding nearest keys\
# range queries.

# they provide both fast key:value lookups and maintain keys in order.

# EXAMPLE - FLIGHT DATABASES 

"""Skip Lists"""

# tHEY STORE DATA MORE EFFICIENTLY. 
# PERFORM operations like insert, delete, search in logarithmic time.

# REALLY GOOD
# in sorted data management
# range queries
# they are easier to implement than TREES

# BAD
# Takes A LOT of memory with all those lists in it
# not widely known.

# FOR QUESTION R-10.1

![MapHierarchy](img/fig102.png)

In [1]:
# R-10.1 
# Give a concrete implementation of the pop method in the context of the
# MutableMapping class, relying only on the five primary abstract methods
# of that class

# The MutableMapping class is an abstract class in Python's 
# collections.abc module that provides a framework for creating mutable
#  mapping types (like dictionaries). To implement the pop method using
#  only the five primary abstract methods of this class, you can
#  leverage the following methods:

# __getitem__(key): Retrieve an item from the mapping by key
# __delitem__(key): Delete an item from the mapping by key.
# __iter__(): Return an iterator over the keys of the mapping.
# __len__(): Return the number of items in the mapping.
# __contains__(key): Check if a key is present in the mapping.


from collections.abc import MutableMapping

class MyMapping(MutableMapping):
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        # Implementing __getitem__ to retrieve an item by key
        return self.data[key]

    def __setitem__(self, key, value):
        # Implementing __setitem__ to set an item by key
        self.data[key] = value

    def __delitem__(self, key):
        # Implementing __delitem__ to delete an item by key
        del self.data[key]

    def __iter__(self):
        # Implementing __iter__ to iterate over the keys
        return iter(self.data)

    def __len__(self):
        # Implementing __len__ to return the number of items
        return len(self.data)

    def __contains__(self, key):
        # Implementing __contains__ to check if a key is present
        return key in self.data

    def pop(self, key, default=None):
        # Implementing pop using the five primary abstract methods
        if key in self:
            value = self[key]
            del self[key]
            return value
        else:
            return default

# Example usage:
my_mapping = MyMapping()
my_mapping['a'] = 1
my_mapping['b'] = 2

print(my_mapping.pop('a', 'Not found'))  # Output: 1
print(my_mapping.pop('c', 'Not found'))  # Output: 'Not found'


1
Not found


In [2]:
# R-10.2 
# Give a concrete implementation of the items() method in the context of
# the MutableMapping class, relying only on the five primary abstract 
# methods of that class. What would its running time be if directly applied to the
# UnsortedTableMap subclass?

# The items() method in the context of the MutableMapping class should
#  return an iterator over the key-value pairs in the mapping. To implement
#  this method using only the five primary abstract methods of the class, you
#  can iterate over the keys using the __iter__() method and retrieve
#  the corresponding values for each key using the __getitem__(key) method.
#  Here's a concrete implementation:

from collections.abc import MutableMapping

class MyMapping(MutableMapping):
    def __init__(self):
        self.data = {}

    def __getitem__(self, key):
        return self.data[key]

    def __setitem__(self, key, value):
        self.data[key] = value

    def __delitem__(self, key):
        del self.data[key]

    def __iter__(self):
        return iter(self.data)

    def __len__(self):
        return len(self.data)

    def __contains__(self, key):
        return key in self.data

    def items(self):
        # Implementing items using the five primary abstract methods
        return ((key, self[key]) for key in self)

# Example usage:
my_mapping = MyMapping()
my_mapping['a'] = 1
my_mapping['b'] = 2

for key, value in my_mapping.items():
    print(f'{key}: {value}')

# Output:
# a: 1
# b: 2


# The items() method implemented in this way has a time complexity
#  of O(n), where n is the number of key-value pairs in the mapping.
#  This is because it needs to iterate over all keys in the mapping 
# using the __iter__() method and retrieve the corresponding values
#  using the __getitem__(key) method for each key.

# If this items() method were directly applied to the UnsortedTableMap
#  subclass, the running time would also be O(n), as the underlying data
#  structure in UnsortedTableMap is typically an unsorted list, and
#  iterating over it and accessing elements by key has linear time
#  complexity.

a: 1
b: 2


In [4]:
# R-10.3 
# Give a concrete implementation of the items() method directly within the
# UnsortedTableMap class, ensuring that the entire iteration runs in O(n)
# time.

# we can iterate over the list of items in _table and yield the key-value pairs

class MapBase(MutableMapping):
    """Our own abstract base class that includes a non public _Item class"""

    # nested _Item class
    class _Item:
        __slots__ = "_key", "_value"
        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key # only compare keys

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
            return self._key < other._key # compare based on keys

class UnsortedTableMap(MapBase):
    """Map implementation using an unordered list."""

    def __init__(self):
        """Make an empty map."""
        self._table = [] # list of Item’s

    def __getitem__(self, k): 
        """Return value associated with key k (raise KeyError if not found)."""
        for item in self._table:
            if k == item._key:
                return item._value
        raise KeyError("Key Error:" + repr(k)) 

    def __setitem__(self, k, v): 
        """Assign value v to key k, overwriting existing value if present."""
        for item in self._table:
            if k == item._key: # Found a match:
                item._value = v # reassign value
                return # and quit
        # did not find match for key
        self._table.append(self._Item(k,v))

    def __delitem__(self, k): 
        """Remove item associated with key k (raise KeyError if not found)."""
        for j in range(len(self._table)):
            if k == self._table[j]. key: # Found a match:
                self._table.pop(j) # remove item 
                return # and quit
        raise KeyError("Key Error:" + repr(k)) 

    def __len__(self):
        """Return number of items in the map."""
        return len(self._table)

    def __iter__(self):
        """Generate iteration of the map s keys."""
        for item in self._table:
            yield item._key # yield the KEY

    def items(self):
        """Generate an iteration of key-value pairs (items) in the map."""
        for item in self._table:
            yield (item._key, item._value)


my_mapping = UnsortedTableMap()
my_mapping['a'] = 1
my_mapping['b'] = 2

for key, value in my_mapping.items():
    print(f'{key}: {value}')


a: 1
b: 2


In [ ]:
# R-10.4 
# What is the worst-case running time for inserting n key-value pairs into an
# initially empty map M that is implemented with the UnsortedTableMap
# class?

# The worst-case running time for inserting n key-value pairs into an initially
#  empty map implemented with the UnsortedTableMap class is O(n^2).
#  This is because, in the worst case, each insertion operation may have to iterate
#  through the entire _table list to check for duplicate keys before adding
#  the new key-value pair.

# Let's break down the worst-case scenario:

# Initially, the _table list is empty.
# When you perform the first insertion, it takes O(1) time because the list is empty.

# For the second insertion, it may have to compare the key with
#  one item in the list (1 comparison).

# For the third insertion, it may have to compare the key with
#  two items in the list (2 comparisons).

# For the fourth insertion, it may have to compare the key with three
#  items in the list (3 comparisons).

# In the worst case, you will perform 1 + 2 + 3 + ... + (n-1) + n = (n*(n+1))/2
#  comparisons, which is equivalent to O(n^2) comparisons and time complexity.

# So, the worst-case running time for inserting n key-value pairs into an initially empty
#  UnsortedTableMap is O(n^2). This is why other data structures like hash tables
#  (e.g., Python's dict) are preferred for scenarios where frequent insertions 
# are required, as they can achieve average-case constant-time insertions.

In [4]:
# R-10.5 
# Reimplement the UnsortedTableMap class from Section 10.1.5, using the
# PositionalList class from Section 7.4 rather than a Python list.


# This was the Positional List class

class DoublyLinkedBase:
    """A base class providing doubly linked list representation"""

    class _Node:

        __slots__ = "_element", "_next", "_prev" # streamline memory usage

        def __init__(self, element, next, prev):
            self._element = element # element of the node
            self._next = next # connect to next node
            self._prev = prev # connect to previous node


    def __init__(self,):
        """Make a new DoublyLinkedList"""
        # These nodes are sentries. They never have elements and 
        # everything happens in between them
        self._header =  self._Node(None, None, None)
        self._trailer =  self._Node(None, None, None)

        # initially no element in the list
        self._header._next = self._trailer
        self._trailer._prev = self._header
        self._size = 0

    def __len__(self):
        return self._size

    def is_empty(self):
        return self._size == 0

    def _insert_between (self, e, predecessor, successor):
        """Add an element between two existing nodes and return the new node"""
        # make a new node
        newest = self._Node(e, predecessor, successor)
        
        # connect to the list
        predecessor._next = newest
        successor._prev = newest
        
        self._size += 1
        return newest

    def _delete_node (self, node):
        """Delete nonsentinel node from the list and return the nodes element"""

        # find where the the given node in the list 
        predecessor = node._prev 
        successor = node._next

        # make new connections, jumping over the given node
        predecessor._next = successor
        successor._prev = predecessor

        # decrease size
        self._size -=1

        element = node._element # get element

        node._prev = node._next = node._element = None # depracate Node

        return element

class PositionalList(DoublyLinkedBase):
    """A sequential container of elements allowing positional access"""

    # ---------------------nested Position Class ---------------------
    class Position:
        """An abstraction representing the location of a single element"""
        def __init__ (self, container, node):
            """This should not be invoked directly"""
            self._container = container
            self._node = node

        def element(self, ):
            return self._node._element

        def __eq__(self, other) -> bool:
            """Return True uf other does not represent the same location"""
            return type(other) is type(self) and other._node is self._node

        def __ne__(self, other):
            """Return True if two other does not represent the same location"""
            return not (self == other) # depending on eq, NICE


    # --------------------utility method ----------------------------------

    def _validate(self,p):
        """Return position's node, or raise appropiate error if invalid"""
        if not isinstance(p , self.Position):
            raise TypeError("p must be a Position type object")
        if p._container is not self:
            raise ValueError("p does not belong to this container")
        if p._node._next is None:           # for deprecated nodes
            # this should not be happening, we have sentries
            raise ValueError("p is no longer valid")
        return p._node

    def _make_position(self, node):
        """Return position instance for a given node, or None if sentinel node"""
        if node is self._header or node is self._trailer:
            return None
        else:
            return self.Position(self, node)

    # ----------------------- accessors --------------------------------

    def first(self):
        """Return the first position in the list or None if empty"""
        return self._make_position(self._header._next)

    def last(self):
        """Return the last position in the list or None if empty"""
        return self._make_position(self._trailer._next)

    def before(self, p):
        """Return the position before the given position p , or None if p is first position"""
        node = self._validate(p)
        return self._make_position(node._prev)

    def after(self, p):
        """Return the position AFTER the given position p , or None if p is last position"""
        node = self._validate(p)
        return self._make_position(node._next)

    def __iter__(self):
        """Generate a forward iteration of the elements of the list"""
        # First position in the list
        cursor = self.first()
        while cursor is not None:
            # yield the element of that position's Node
            yield cursor.element()
            cursor = self.after(cursor)

    # ------------------------ Mutators --------------------------------

    # override inherited version to return position, rather than Node
    def _insert_between(self, e, predecessor, successor):
        """Add element between existing nodes and return new position """
        node = super()._insert_between(e, predecessor, successor) 
        return self._make_position(node)

    def add_first(self,e):
        """Insert element e at the front of the list and return new position"""
        return self._insert_between(e, self._header, self._header._next)

    def add_last(self,e):
        """Insert element e at the back of the list and return new position"""
        return self._insert_between(e, self._trailer._prev, self._trailer)

    def add_before(self, p, e):
        """Insert element e into list before Position p and return new position"""
        original = self._validate(p)
        return self._insert_between(e, original._prev, original)

    def add_after(self, p,e):
        """Insert element e into list after Position p and return new position"""
        original = self._validate(p)
        return self._insert_between(e, original, original._next)

    def delete(self, p):
        """Remove and return element e at position p"""
        original = self._validate(p)
        return self._delete_node(original)

    def replace(self, p , e):
        """Replace the element at Position p with e
        
        Return the element formerly at Position p"""

        original = self._validate(p)
        old_value = original._element # temporarily store old element
        original._element = e # replace with new element
        return old_value # return the old element value


# here is the answer


class UnsortedTableMap:
    """Map implementation using an unordered list."""

    def __init__(self):
        """Make an empty map."""
        self._table = PositionalList()

    def __getitem__(self, k):
        """Return value associated with key k (raise KeyError if not found)."""
        for key, value in self._table:
            if k == key:
                return value
        raise KeyError("Key Error: " + repr(k))

    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present."""
        for position, (key, value) in self._table:
            if k == key:
                position.element = (k, v)  # Replace the existing key-value pair
                return
        # If the key is not found, add it to the list
        self._table.add_last((k, v))

    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found)."""
        for position, (key, value) in self._table:
            if k == key:
                self._table.delete(position)
                return
        raise KeyError("Key Error: " + repr(k))

    def __len__(self):
        """Return the number of items in the map."""
        return len(self._table)

    def __iter__(self):
        """Generate iteration of the map's keys."""
        for key, _ in self._table:
            yield key

    def items(self):
        """Generate an iteration of key-value pairs (items) in the map."""
        for key, value in self._table:
            yield (key, value)


map = UnsortedTableMap()

map["a"] = 1

print(map)
print(map["a"])
print([elem for elem in map.items()])


1
[('a', 1)]


In [ ]:
# TODO

# R-10.6 
# Which of the hash table collision-handling schemes could tolerate a load
# factor above 1 and which could not?

# Certainly, let's focus on these specific collision-handling schemes: Separate 
# Chaining, Open Addressing (including Linear Probing), Robin Hood Hashing, Double
#  Hashing, and Quadratic Probing, and discuss their ability to tolerate a load 
# factor above 1:
# 

# Separate Chaining:
# 
# Tolerance to Load Factor Above 1: Separate chaining can tolerate a 
# load factor above 1 relatively well. As the load factor increases, the 
# average length of the linked lists (used to store colliding elements) increases, which
#  can lead to slower lookup times but doesn't necessarily cause a fundamental 
# failure of the data structure. It can handle load factors above 1 without a
#  significant issue, but performance may degrade as the load factor becomes much higher.

# Open Addressing (Including Linear Probing, Double Hashing, and Quadratic Probing):
# 
# Tolerance to Load Factor Above 1: 
# 
# Open addressing schemes 
# (including Linear Probing, Double Hashing, and Quadratic Probing)
#  They may have difficulty 
# tolerating a load factor above 1, especially when the load factor is significantly
#  greater than 1. Open addressing relies on probing for the next available slot when
#  a collision occurs. A higher load factor increases the likelihood of clustering
#  (groups of keys clustered together), leading to a higher number of collisions and
#  potentially significant performance degradation.

# Open addressing requires that the load
# factor is always at most 1 and that items are stored directly in the cells 
# of the bucket array itself

# Robin Hood Hashing:
# 
# Tolerance to Load Factor Above 1: Robin Hood hashing, which is a variant of
#  open addressing, aims to redistribute keys to minimize differences in probing 
# distances. It can tolerate a load factor slightly above 1 and is designed to
#  maintain a relatively balanced distribution of keys. However, as the load factor
#  increases significantly, it may still face challenges due to clustering and
# increased probing distances.

# Double Hashing:
# 
# Tolerance to Load Factor Above 1: Double hashing, another open addressing 
# scheme, can handle a load factor slightly above 1 but may face challenges
#  with a significantly higher load factor. It relies on a secondary hash 
# function to determine the step size for probing. Careful design and choice 
# of hash functions can help improve its tolerance to higher load factors.

# Quadratic Probing:
# 
# Tolerance to Load Factor Above 1: Quadratic probing, like other open addressing 
# techniques, may not handle a load factor significantly above 1 effectively due 
# to clustering. Its performance can degrade as the load factor increases beyond
#  a certain threshold.

# In summary, separate chaining is relatively robust and can tolerate a load 
# factor above 1 without fundamental issues. Open addressing schemes, including
#  linear probing, double hashing, and quadratic probing, may have difficulty 
# handling significantly higher load factors and can experience performance 
# degradation due to clustering. Robin Hood hashing and double hashing can handle
#  load factors slightly above 1 with appropriate design and probing strategies.
#  The choice of collision-handling scheme should consider the expected load factor 
# and the trade-offs between memory usage and performance.

# Summary from book:

# In the hash table schemes described thus far, it is important that the load factor,
# λ = n/N, be kept below 1. With separate chaining, as λ gets very close to 1, the
# probability of a collision greatly increases, which adds overhead to our operations,
# since we must revert to linear-time list-based methods in buckets that have collisions.
#  Experiments and average-case analyses suggest that we should maintain
# λ < 0.9 for hash tables with separate chaining.

# With open addressing, on the other hand, as the load factor λ grows beyond 0.5
# and starts approaching 1, clusters of entries in the bucket array start to grow as well.
# These clusters cause the probing strategies to “bounce around” the bucket array for
# a considerable amount of time before they find an empty slot. In Exercise C-10.36,
# we explore the degradation of quadratic probing when λ ≥0.5. Experiments suggest
#  that we should maintain λ < 0.5 for an open addressing scheme with linear
# probing, and perhaps only a bit higher for other open addressing schemes (for 
# example, Python’s implementation of open addressing enforces that λ < 2/3)


In [2]:
# R-10.7 
# 
# Our Position classes for lists and trees support the __eq__ method so that
# two distinct position instances are considered equivalent if they refer to the
# same underlying node in a structure. For positions to be allowed as keys
# in a hash table, there must be a definition for the hash method that
# is consistent with this notion of equivalence. Provide such a hash
# method.

# To provide a `hash` method for positions that is consistent with the notion of equivalence
#  where two distinct position instances are considered equivalent if they refer to
#  the same underlying node, you can use the `id()` function to hash the
#  underlying node. The `id()` function returns a unique identifier for an object in
#  Python. Here's how you can implement the `hash` method for positions:

class Position:
    """An abstraction representing the location of a single element"""
    def __init__ (self, container, node):
        """This should not be invoked directly"""
        self._container = container
        self._node = node
    
    def element(self, ):
        return self._node._element
    
    def __eq__(self, other) -> bool:
        """Return True uf other does not represent the same location"""
        return type(other) is type(self) and other._node is self._node
    
    def __ne__(self, other):
        """Return True if two other does not represent the same location"""
        return not (self == other) # depending on eq, NICE

    def __hash__(self):
        """
        Return a hash value based on the unique identifier of the underlying node.
        """
        return hash(id(self._node))

# In this implementation, the `hash()` method uses the `id(self._node)` to 
# obtain a unique identifier for the underlying node, and then it hashes this unique
#  identifier. This ensures that two distinct position instances that refer to the
#  same underlying node will have the same hash value, satisfying the requirement for
#  positions to be allowed as keys in a hash table.

# Here's an example of how you can use this `hash` method with positions:

# Make two position instances referring to the same node

container = {}
node = 3

position1 = Position(container, node)
position2 = Position(container, node)

# Check if they are equivalent
print(position1 == position2)  # True

# Use them as keys in a dictionary
position_dict = {}
position_dict[position1] = "Value 1"
position_dict[position2] = "Value 2"

# Check the dictionary
print(position_dict[position1])  # Output: Value 2
print(position_dict[position2])  # Output: Value 2

# In this example, both `position1` and `position2` refer to the same node, and 
# their `hash` values are consistent, allowing them to be used as keys in a dictionary.

True
Value 2
Value 2


In [ ]:
# R-10.8 
# What would be a good hash code for a vehicle identification number that
# is a string of numbers and letters of the form “9X9XX99X9XX999999,”
# where a “9” represents a digit and an “X” represents a letter?

# A good hash code for a vehicle identification number (VIN) that is in the 
# form "9X9XX99X9XX999999," where "9" represents a digit and "X" represents a
#  letter, can be generated by considering both the numeric and alphabetic
#  characters. Here's one way to generate a hash code for such a VIN:

# Convert the alphabetic characters (X) to their corresponding numeric values.

# Concatenate all the numeric and converted alphabetic values into a single string.

# Compute the hash code of the resulting string using a suitable hash function.

def vin_hash(vin):
    # Define a dictionary to map alphabetic characters to numeric values
    char_to_num = {'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15,
                   'G': 16, 'H': 17, 'J': 18, 'K': 19, 'L': 20, 'M': 21,
                   'N': 22, 'P': 23, 'R': 24, 'S': 25, 'T': 26, 'U': 27,
                   'V': 28, 'W': 29, 'X': 30, 'Y': 31, 'Z': 32}

    # Convert alphabetic characters to numeric values and concatenate
    numeric_vin = ''.join(str(char_to_num[c]) if c in char_to_num else c for c in vin)

    # Compute the hash code of the resulting string
    hash_code = hash(numeric_vin)

    return hash_code

# This vin_hash function converts alphabetic characters to numeric values based 
# on the dictionary char_to_num, then concatenates all the characters to form
#  a numeric string, and finally computes the hash code of the resulting string 
# using Python's built-in hash function.

# Keep in mind that the quality of the hash code may depend on the distribution of 
# VINs you expect to encounter in practice. If there are specific constraints
#  or patterns in the VINs, you might need to adjust the hashing approach accordingly.

In [ ]:
# R-10.9 
# Draw the 11-entry hash table that results from using the hash function,
# h(i) = ( 3i+ 5) mod 11, to hash the keys 12, 44, 13, 88, 23, 94, 11, 39, 20,
# 16, and 5, assuming collisions are handled by chaining.

# To draw the 11-entry hash table resulting from using the hash function h(i) = (3i + 5) \mod 11
# h(i)=(3i+5)mod11 to hash the given keys (12, 44, 13, 88, 23, 94, 11, 39, 20, 16, and 5) with
#  collisions handled by chaining, you can calculate the hash values for each key and place
#  them in the appropriate slot in the hash table.

# Key 12 hashes to h(12) = (3 * 12 + 5) \ mod 11 = 8, so it is placed in Index 8.
# Key 44 hashes to h(44) = (3 * 44 + 5) \ mod 11 = 5, so it is placed in Index 5.
# Key 13 hashes to h(13) = (3 * 13 + 5) \ mod 11 = 0, so it is placed in Index 0.
# Key 88 hashes to h(88) = (3 * 88 + 5) \ mod 11 = 5, so it is placed in Index 5.
# Key 23 hashes to h(23) = (3 * 23 + 5) \ mod 11 = 10, so it is placed in Index 10.
# Key 94 hashes to h(94) = (3 * 94 + 5) \ mod 11 = 1, so it is placed in Index 1.
# Key 11 hashes to h(11) = (3 * 11 + 5) \ mod 11 = 5, so it is placed in Index 5.
# Key 39 hashes to h(39) = (3 * 39 + 5) \ mod 11 = 7, so it is placed in Index 7.
# Key 20 hashes to h(20) = (3 * 20 + 5) \ mod 11 = 10, so it is placed in Index 10.
# Key 16 hashes to h(16) = (3 * 16 + 5) \ mod 11 = 9, so it is placed in Index 9.
# Key 5 hashes to h(5) = (3 * 5 + 5) \ mod 11 = 9, so it is placed in Index 9. 

# Index 0: [13]
# Index 1: [94]
# Index 2: []
# Index 3: []
# Index 4: []
# Index 5: [44, 88, 11]
# Index 6: []
# Index 7: [39]
# Index 8: [12]
# Index 9: [16, 5]
# Index 10: [20, 23]

In [ ]:
# R-10.10 
# What is the result of the previous exercise, assuming 
# collisions are handled by linear probing?

# Lets insert the keys one by one.

# key 12 - index 8
# key 44 - index 5
# key 13 - index 0 
# key 88 - index 5  - not empty - index 6
# key 23 - index 10 
# key 94 - index 1
# key 11 - index 5 - not empty - index 6 - not empty - index 7
# key 39 - index 7 - not empty - index 8 - not empty - index 9
# key 20 - index 10 - not empty - index 0 - not empty - index 1 - not empty - index 2
# key 16 - index 9 - not empty - index 0 - not empty - index 1 - not empty 
#                           - index 2 - not empty - index 3
# key 5 - index 9   - not empty - index 0 - not empty - index 1 - not empty 
#                           - index 2 - not empty - index 3 - not empty - index 4

# Index 0: [13] 
# Index 1: [94]
# Index 2: [20]
# Index 3: [16]
# Index 4: [5]
# Index 5: [44]
# Index 6: [88]
# Index 7: [11]
# Index 8: [12]
# Index 9: [39]
# Index 10: [23]

In [2]:
# R-10.11 
# Show the result of Exercise R-10.9, assuming collisions are handled by
# quadratic probing, up to the point where the method fails.

def calculate(seq):
    result = []
    for elem in seq:
        result.append(((elem * 3)+5) % 11)
    return result

calculate([12, 44, 13, 88, 23, 94, 11, 39, 20, 16, 5])
# [8, 5, 0, 5, 8, 1, 5, 1, 10, 9, 9]

# Quadratic probing is a collision resolution technique used in hash tables to find
#  an available slot when a collision occurs. When multiple keys hash to the same
#  index, quadratic probing attempts to find the next available index by using a
#  quadratic sequence of indices, rather than simply linearly probing (i.e., checking
#  the next index and then the one after that).

# Here's how quadratic probing works:

# 1) Calculate the initial hash index for the key.

# 2) If the calculated index is already occupied by another key, start a quadratic
#  sequence of probes. The sequence is typically generated using the formula 
# i^2i  for increasing values of ii. So, you start with i = 1 for the first
#  probe, i = 2 for the second probe, i = 3 for the third probe, and so on.

# 3) Calculate the new index by adding (i^2) to the original index.

# 4) Check if the new index is within the bounds of the hash table. If it
#  is, and the slot is empty, insert the key at that index. If it's occupied, continue
#  the quadratic probing sequence with the next i.

# 5) Repeat steps 3 and 4 until an empty slot is found or the entire hash table is 
# searched (in which case, it is considered full).

# here are the results for quadratic probing

# Key 12 hashes to h(12) = 8h(12)=8 and is placed at Index 8.

# Key 44 hashes to h(44) = 5h(44)=5, but Index 5 is occupied by Key 23. Quadratic 
# probing moves to Index 9, where Key 44 is placed.

# Key 13 hashes to h(13) = 0h(13)=0 and is placed at Index 0.

# Key 88 hashes to h(88) = 4h(88)=4, but Index 4 is occupied by Key 23. Quadratic
#  probing moves to Index 9 (occupied by Key 44), then to Index 0, where Key 88 is placed.

# Key 23 hashes to h(23) = 10h(23)=10, but Index 10 is occupied by Key 12. Quadratic probing
#  moves to Index 0 (occupied by Key 13), then to Index 3, where Key 23 is placed.

# Key 94 hashes to h(94) = 1h(94)=1 and is placed at Index 1.

# Key 11 hashes to h(11) = 5h(11)=5, but Index 5 is occupied by Key 23. Quadratic
#  probing moves to Index 9 (occupied by Key 44), then to Index 0 (occupied by
#  Key 13), and finally to Index 8, where Key 11 is placed.

# Key 39 hashes to h(39) = 7h(39)=7 and is placed at Index 7.

# Key 20 hashes to h(20) = 10h(20)=10, but Index 10 is occupied by Key 23. Quadratic
#  probing moves to Index 0 (occupied by Key 13), then to Index 3 (occupied by Key
#  23), and finally to Index 8 (occupied by Key 11). It fails to find an empty slot.

# Key 16 hashes to h(16) = 9h(16)=9, but Index 9 is occupied by Key 39. Quadratic probing
#  moves to Index 0 (occupied by Key 13), then to Index 3 (occupied by Key 23), then
#  to Index 8 (occupied by Key 11), and finally to Index 1, where Key 16 is placed.

# Key 5 hashes to h(5) = 9h(5)=9, but Index 9 is occupied by Key 39. Quadratic probing 
# moves to Index 0 (occupied by Key 13), then to Index 3 (occupied by Key 23), then to
#  Index 8 (occupied by Key 11), then to Index 1 (occupied by Key 16), and finally to
#  Index 4, where Key 5 is placed.


# Index 0: 13
# Index 1: 94
# Index 2: 
# Index 3: 23
# Index 4: 5
# Index 5: 
# Index 6: 
# Index 7: 39
# Index 8: 11
# Index 9: 16
# Index 10: 12


[8, 5, 0, 5, 8, 1, 5, 1, 10, 9, 9]

In [ ]:
# TODO

# R-10.12 
# What is the result of Exercise R-10.9 when collisions are handled by 
# double hashing using the secondary hash function h′(k) = 7 − (k mod 7)?

In [ ]:
# R-10.13 
# What is the worst-case time for putting n entries in an initially empty hash
# table, with collisions resolved by chaining? What is the best case?

# all hashes collide - worst case

# Worst-case time: In the worst-case scenario, all n entries hash to the same 
# index in the table, resulting in a long chain of collisions. This can lead to
#  the hash table essentially degenerating into a linked list. In this worst-case 
# scenario, the time complexity for putting n entries is O(n), as each insertion
#  into the chain takes O(1) time, but there are n entries to insert.

# Best-case time: In the best-case scenario, the hash function distributes the n 
# entries evenly across the hash table, and there are no collisions. Each insertion 
# into the table takes O(1) time in the best case, as there are no collisions to 
# resolve. Therefore, the best-case time complexity for putting n entries is O(n), which
#  is the same as the worst-case when using chaining.

# It's important to note that the actual performance of a hash table can vary depending
#  on factors such as the quality of the hash function and the load factor (the ratio
#  of the number of entries to the size of the table). In practice, a well-designed
#  hash function and proper resizing of the table can help minimize collisions and 
# achieve good average-case performance.


# FOR QUESTION R-10.14

![MapHierarchy](img/fig106.png)

In [ ]:
# TODO: Work

# R-10.14 
# Show the result of rehashing the hash table shown i  n Figure 10.6 into a
# table of size 19 using the new hash function h(k) = 3k mod 17.

In [ ]:
# R-10.15 
# Our HashMapBase class maintains a load factor λ ≤0.5. Reimplement
# that class to allow the user to specify the maximum load, and adjust the
# concrete subclasses accordingly.

# This was the original implementation 

from random import randrange

class MapBase(MutableMapping):
    """Our own abstract base class that includes a non public _Item class"""

    # nested _Item class
    class _Item:
        __slots__ = "_key", "_value"
        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key      # only compare keys

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
             return self._key < other._key      # compare based on keys

class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression"""
    def __init__(self, cap = 11, p = 109345121):
        """Make an empty hash-table map"""
        self._table = cap * [None]
        self._n = 0                         # Number of entries in the map
        self._prime = p                     # prime for MAD compression
        self._scale = 1 + randrange(p-1)    # scale from 1 to p-1 for MAD
        self._shift = randrange(p)          # shift from 0 to p-1 for MAD

    def _hash_function(self,k):
        return (hash(k) *  self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self,k):
        j = self._hash_function(k)
        return self._bucket_getitem(j,k)

    def __setitem__(self, k, v) :
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)               # subroutine maintains self._n
        if self._n > len(self._table) // 2:         # keep load factor <= 0.5
            self._resize(2*len(self._table) - 1)    # number 2^x - 1 often prime

    def _resize(self, c):               # resize bucket array to capacity c
        old = list(self.items())        # use iteration to record existing items
        self._table = c * [None]        # reset the table to desired capacity
        self._n = 0                     # n recomputed during subsequent adds
        for (k,v) in old:       
            self[k] = v                  # reinsert old key value pair

    def __delitem__(self, k ):
        j = self._hash_function(k)
        self._bucket_delitem(j,k)       # may Raise KeyError
        self._n -= 1

# Here is the answer
# Only the setitem method will change
# Additinally we can define load factor function to make code cleaner.

class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression"""

    def __init__(self, cap=11, max_load=0.5, p=109345121):
        """Make an empty hash-table map"""
        self._table = cap * [None]
        self._n = 0  # Number of entries in the map
        self._prime = p  # prime for MAD compression
        self._scale = 1 + randrange(p - 1)  # scale from 1 to p-1 for MAD
        self._shift = randrange(p)  # shift from 0 to p-1 for MAD
        # setup instance variable
        self._max_load = max_load  # Maximum load factor

    def _hash_function(self, k):
        return (hash(k) * self._scale + self._shift) % self._prime % len(self._table)

    def _load_factor(self):
        return self._n / len(self._table)

    def __setitem__(self, k, v):
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)  # subroutine maintains self._n
        if self._load_factor() > self._max_load:
            self._resize(2 * len(self._table) - 1)  # number 2^x - 1 often prime

    def _resize(self, c):  # resize bucket array to capacity c
        old = list(self.items())  # use iteration to record existing items
        self._table = c * [None]  # reset the table to desired capacity
        self._n = 0  # n recomputed during subsequent adds
        for (k, v) in old:
            self[k] = v  # reinsert old key-value pairs

    def __delitem__(self, k):
        j = self._hash_function(k)
        self._bucket_delitem(j, k)  # may Raise KeyError
        self._n -= 1


In [ ]:
# R-10.16 
# Give a pseudo-code description of an insertion into a hash table that uses
# quadratic probing to resolve collisions, assuming we also use the trick of
# replacing deleted entries with a special “deactivated entry” object

# lets remember quadratic probing first

# Quadratic probing is a collision resolution technique used in hash tables to find
#  an available slot when a collision occurs. When multiple keys hash to the same
#  index, quadratic probing attempts to find the next available index by using a
#  quadratic sequence of indices, rather than simply linearly probing (i.e., checking
#  the next index and then the one after that).

# Here's how quadratic probing works:

# 1) Calculate the initial hash index for the key.

# 2) If the calculated index is already occupied by another key, start a quadratic
#  sequence of probes. The sequence is typically generated using the formula 
# i^2i  for increasing values of i. So, you start with i = 1 for the first
#  probe, i = 2 for the second probe, i = 3 for the third probe, and so on.

# 3) Calculate the new index by adding (i^2) to the original index.

# 4) Check if the new index is within the bounds of the hash table. If it
#  is, and the slot is empty, insert the key at that index. If it's occupied, continue
#  the quadratic probing sequence with the next i.

# 5) Repeat steps 3 and 4 until an empty slot is found or the entire hash table is 
# searched (in which case, it is considered full).

# here is the code and a description of an insertion into a hash table that uses
#  quadratic probing to resolve collisions and replaces deleted entries with a 
# special "deactivated entry" object:

class HashTable:
    """Construct a hash table upon Python lists - dynamic arrays"""
    def __init__(self, size):
        # Initialize a hash table with the given size
        self.size = size
        self.table = [None] * size

    def hash_function(self, key):
        # Calculate the initial hash index for the key
        return hash(key) % self.size

    def quadratic_probe(self, index, i):
        # Calculate the new index using quadratic probing
        return (index + i * i) % self.size

    def insert(self, key, value):
        index = self.hash_function(key)
        i = 0

        while i < self.size:
            new_index = self.quadratic_probe(index, i)

            if self.table[new_index] is None or self.table[new_index].is_deactivated():
                # If the slot is empty or contains a deactivated entry, insert the new entry
                self.table[new_index] = Entry(key, value)
                return True
            elif self.table[new_index].is_deleted():
                # If the slot contains a deleted entry, replace it with the new entry
                self.table[new_index] = Entry(key, value)
                return True
            else:
                # Collision occurred, continue probing
                i += 1

        # If the loop completes without finding an empty slot, the table is full
        return False

    def delete(self, key):
        index = self.hash_function(key)
        i = 0

        while i < self.size:
            new_index = self.quadratic_probe(index, i)

            if self.table[new_index] is None:
                # Key not found
                return False
            elif self.table[new_index].key == key:
                # Mark the entry as deleted
                self.table[new_index].mark_as_deleted()
                return True
            else:
                # Collision occurred, continue probing
                i += 1

        # Key not found
        return False

class Entry:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.deleted = False

    def is_deleted(self):
        return self.deleted

    def is_deactivated(self):
        return self.deleted

    def mark_as_deleted(self):
        self.deleted = True

# Description: 

# HashTable is the class representing the hash table with quadratic probing.

# hash_function calculates the initial hash index for the key. - Hash method returns an INTEGER

# quadratic_probe calculates the new index using quadratic probing.

# insert method inserts a key-value pair into the hash table, using quadratic probing
#  to resolve collisions. If a slot is empty or contains a deactivated entry, it
#  inserts the new entry. If it encounters a deleted entry, it replaces it with the new entry.

# delete method marks an entry as deleted if it matches the specified key.

# Entry is a class representing entries in the hash table. Each entry contains a 
# key, value, and a flag to indicate if it's deleted or deactivated. Deleted entries
#  are treated as deactivated.

# This approach ensures that deleted entries are replaced with a 
# special "deactivated entry" object, allowing them to be reused for future insertions.

In [13]:
# R-10.17 
# 
# Modify our ProbeHashMap to use quadratic probing.

# Here is our subclasses.

from collections.abc import MutableMapping

class MapBase(MutableMapping):
    """Our own abstract base class that includes a non public _Item class"""

    # nested _Item class
    class _Item:
        __slots__ = "_key", "_value"
        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key      # only compare keys

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
             return self._key < other._key      # compare based on keys

class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression"""
    def __init__(self, cap = 11, p = 109345121):
        """Make an empty hash-table map"""
        self._table = cap * [None]
        self._n = 0                         # Number of entries in the map
        self._prime = p                     # prime for MAD compression
        self._scale = 1 + randrange(p-1)    # scale from 1 to p-1 for MAD
        self._shift = randrange(p)          # shift from 0 to p-1 for MAD

    def _hash_function(self,k):
        return (hash(k) *  self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self,k):
        j = self._hash_function(k)
        return self._bucket_getitem(j,k)

    def __setitem__(self, k, v) :
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)               # subroutine maintains self._n
        if self._n > len(self._table) // 2:         # keep load factor <= 0.5
            self._resize(2*len(self._table) - 1)    # number 2^x - 1 often prime

    def _resize(self, c):               # resize bucket array to capacity c
        old = list(self.items())        # use iteration to record existing items
        self._table = c * [None]        # reset the table to desired capacity
        self._n = 0                     # n recomputed during subsequent adds
        for (k,v) in old:       
            self[k] = v                  # reinsert old key value pair

    def __delitem__(self, k ):
        j = self._hash_function(k)
        self._bucket_delitem(j,k)       # may Raise KeyError
        self._n -= 1

class ProbeHashMap(HashMapBase):
    """Hash map implemented with linear probing for collision resolution."""
    _AVAIL = object() # sentinel object that marks locations of previous deletions

    def _is_avaliable(self):
        """Return True if index j is avaliable in table."""
        return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL

    def _find_slot(self, j ,k):
        r"""
        Search for key k in bucket at index j.
        Return (success, index) tuple, described as follows:
        If match was found, success is True and index denotes its location.
        If no match found, success is False and index denotes first available slot.
        """
        firstAvail = None
        while True:
            if self._is_avaliable(j):
                if firstAvail is None:
                    firstAvail = j                          # mark this as first avail
                if self._table[j] is None:
                    return (False, firstAvail)              # search has failed
            elif k == self._table[j]._key:
                return (True, j)                            # found a match
            j = (j + 1) % len(self._table)                  # keep looking (cyclically)

    def bucket_getitem(self, j, k): 
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError( "Key Error:" + repr(k))         # no match found
        return self._table[s]._value

    def bucket_setitem(self, j, k, v): 
        found, s = self._find_slot(j, k)
        if not found:
            self._table[s] = self._Item(k,v)                # insert new item
            self._n += 1                                    # size has increased
        else:
            self._table[s]._value = v                       # overwrite existing

    def bucket_delitem(self, j, k): 
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError( "Key Error:" + repr(k))         # no match found
        self._table[s] = ProbeHashMap._AVAIL                # mark as vacated

    def __iter__(self):
        for j in range(len(self. table)):                   # scan entire table
            if not self._is_avaliable(j):
                yield self._table[j]._key


# answer - only _find_slot will change

class ProbeHashMap(HashMapBase):
    """Hash map implemented with linear probing for collision resolution."""
    _AVAIL = object() # sentinel object that marks locations of previous deletions

    def _is_avaliable(self, j):
        """Return True if index j is avaliable in table."""
        return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL

    def _find_slot(self, j, k):
        r"""
        Search for key k in bucket at index j using quadratic probing.
        Return (success, index) tuple, described as follows:
        If match was found, success is True and index denotes its location.
        If no match is found, success is False and index denotes the first available slot.
        """
        first_avail = None
        count = 0  # Counter for quadratic probing

        while True:
            if self._is_avaliable(j):
                if first_avail is None:
                    first_avail = j  # Mark this as the first available slot
                if self._table[j] is None:
                    return (False, first_avail)  # Search has failed
            elif k == self._table[j]._key:
                return (True, j)  # Found a match

            # Quadratic probing: j + count^2
            count += 1
            j = (j + count**2) % len(self._table)  # Keep looking (cyclically)

    def bucket_getitem(self, j, k): 
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError( "Key Error:" + repr(k))         # no match found
        return self._table[s]._value

    def bucket_setitem(self, j, k, v): 
        found, s = self._find_slot(j, k)
        if not found:
            self._table[s] = self._Item(k,v)                # insert new item
            self._n += 1                                    # size has increased
        else:
            self._table[s]._value = v                       # overwrite existing

    def bucket_delitem(self, j, k): 
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError( "Key Error:" + repr(k))         # no match found
        self._table[s] = ProbeHashMap._AVAIL                # mark as vacated

    def __iter__(self):
        for j in range(len(self. table)):                   # scan entire table
            if not self._is_avaliable(j):
                yield self._table[j]._key

In [ ]:
# R-10.18 
# 
# Explain why a hash table is not suited to implement a sorted map

# First of all, what is a sorted map?

# The traditional map ADT allows a user to look up the value associated with a given
# key, but the search for that key is a form known as an exact search.

# Map ADT does not provide any way to get a list of all events ordered by the time at
# which they occur, or to search for which event occurred closest to a particular time.
# In fact, the fast performance of hash-based implementations of the map ADT relies
# on the intentionally scattering of keys that may seem very “near” to each other in
# the original domain, so that they are more uniformly distributed in a hash table.

# here are some methods for it
# M.find min( ): Return the (key,value) pair with minimum key (or None, if map is empty).
# M.find max( ): Return the (key,value) pair with maximum key (or None, if map is empty).

# M.find lt(k): Return the (key,value) pair with the greatest key that
# is strictly less than k (or None, if no such item exists). 
# M.find le(k): Return the (key,value) pair with the greatest key that
# is less than or equal to k (or None, if no such item exists).

# M.find gt(k): Return the (key,value) pair with the least key that is
# strictly greater than k (or None, if no such item exists). 
# M.find ge(k): Return the (key,value) pair with the least key that is
# greater than or equal to k (or None, if no such item). 

# M.find range(start, stop): Iterate all (key,value) pairs with start <= key < stop.
# If start is None, iteration begins with minimum key; if
# stop is None, iteration concludes with maximum key.

# iter(M): Iterate all keys of the map according to their natural
# order, from smallest to largest.
# reversed(M):

# the answer

# A hash table is not well-suited to implement a sorted map for several reasons:

""" Lack of Natural Ordering:""" 
# Hash tables are inherently unordered data structures. They
#  store key-value pairs based on the hash values of keys, and this arrangement 
# doesn't provide any inherent order. In contrast, a sorted map requires that the
#  elements be stored in a specific order based on their keys.

"""Inefficient for Range Queries:"""
# #  Sorted maps often require efficient support for range
#  queries, such as finding all keys within a given range. Hash tables do not provide
#  a natural way to perform such queries without iterating over all entries, which can
#  be inefficient.

"""Extra Overhead:""" 
# Implementing a sorted map using a hash table would require additional 
# data structures or operations to maintain the sorted order of keys. This added 
# complexity can lead to increased memory usage and slower performance compared
#  to data structures explicitly designed for sorting, like balanced search
#  trees (e.g., AVL trees or Red-Black trees).

""" Difficulty in Maintaining Sorting Order:""" 
# While you could potentially sort the keys
#  of a hash table after each insertion, this would be inefficient for large data 
# sets. Additionally, maintaining the sorted order during insertions, deletions, and
#  updates would be a challenging task and could lead to performance degradation.

"""Loss of Constant-Time Access:""" 
# One of the primary advantages of hash tables is their
#  constant-time average-case access time (O(1)). Implementing sorting and maintaining
#  it would likely result in a loss of this constant-time guarantee for certain operations.

# For implementing a sorted map, data structures like balanced search trees (e.g., Red-Black
#  trees or AVL trees) or skip lists are better-suited choices. These structures 
# inherently maintain order, support efficient range queries, and provide a natural
#  way to insert, delete, and retrieve elements while keeping the elements sorted.


In [ ]:
# R-10.19 
# Describe how a sorted list implemented as a doubly linked list could be
# used to implement the sorted map ADT.

# A sorted list implemented as a doubly linked list can be used as a foundational data
#  structure to implement the sorted map ADT (Abstract Data Type). 
# Here's how it can be achieved:

"""Data Structure for the Sorted List:"""

"""Doubly Linked List:""" 
# Use a doubly linked list to maintain the elements of 
# the sorted list. Each node of the linked list contains a key-value pair.

"""Sorted Order:""" 
# Ensure that the elements in the linked list are arranged in
#  sorted order based on the keys. The keys should be used as the criteria 
# for sorting.

"""Operations for Implementing Sorted Map:"""

"""Insertion:""" 
# When inserting a new key-value pair into the sorted map, you can perform
#  an insertion operation into the sorted list. To maintain the sorted order, find
#  the correct position in the linked list for the new key and insert it there. 
# This may involve traversing the list to find the appropriate location.

"""Deletion:""" 
# To delete a key-value pair with a specific key, search for the key in the linked 
# list and remove the corresponding node if found.

"""Search:""" 
# To search for a key in the sorted map, traverse the linked 
# list in sorted order, comparing the keys at each step until you find the key 
# you are looking for.

"""Range Queries:""" 
# Since the elements are already sorted, implementing range queries (e.g., finding all
#  key-value pairs within a specified range of keys) becomes straightforward. You can 
# traverse the list and select the elements that fall within the desired range.

"""Iteration in Sorted Order:""" 
# When iterating through the sorted map, you can start at the head or tail of the linked 
# list, depending on whether you want to iterate in ascending or descending order. Then, simply 
# traverse the linked list, visiting each key-value pair in sorted order.

"""Complexity Analysis:"""

"""Insertion and Deletion:""" 
# In the worst case, both insertion and deletion operations may require traversing the entire 
# linked list, resulting in O(n) time complexity, where n is the number of elements in the sorted map.

"""Search:""" 
# Searching for a specific key in the sorted map also has a worst-case time complexity of O(n)
#  due to the need to traverse the list.

"""Range Queries and Iteration:"""
#  Both range queries and iterating in sorted order are efficient operations, as they involve 
# traversing only the relevant portion of the linked list.

"""Advantages:"""

# Simplicity: Implementing a sorted map with a sorted list is conceptually straightforward.
# Range Queries: It efficiently supports range query operations due to the inherent sorting.
# Iteration in Order: Elements can be efficiently iterated in sorted order.

"""Disadvantages:"""

# Inefficient for Insertions and Deletions: The worst-case time complexity for insertions
#  and deletions is O(n), which can be inefficient for large datasets.
# Space Overhead: Each element requires extra space for the doubly linked list pointers.

# In summary, a sorted list implemented as a doubly linked list is a viable option for
#  implementing a sorted map ADT, but it may not be the most efficient choice for scenarios
#  with frequent insertions and deletions. For those cases, balanced search trees 
# (e.g., Red-Black trees or AVL trees) may provide better performance.


In [ ]:
# R-10.20 
# What is the worst-case asymptotic running time for performing n deletions
# from a SortedTableMap instance that initially contains 2n entries?

# n deletions.
# for each deletion, the time complexity can be o(n) in the worst case.
# this is just for traversing through.

# FOR QUESTION 10.21

![Analysis of SortedTableMap](img/table103.png)

In [ ]:
# R-10.21 
# Consider the following variant of the find index method from Code 
# Fragment 10.8, in the context of the SortedTableMap class:

def _find_index(self, k, low, high): 
    """In contrast, the provided variant of _find_index recursively calls
     itself with a new range that includes both the upper and lower halves
      when the key at mid is less than k. This means that it might not
       return the index of the first occurrence of k but rather the index
    of the last occurrence of k."""
    if high < low:
        return high + 1
    else:
        mid = (low + high) // 2
        if self._table[mid]._key < k:
            return self._find_index(k, mid + 1, high)
        else:
            return self._find_index(k, low, mid -1)

# Does this always produce the same result as the original version? Justify your answer.

# This was the original version of the class

class SortedTableMap(MapBase):
    """
    Map implementation using a sorted table.
    """
    # nonpublic behaviors
    def _find_index(self, k, low, high):
        """
        Return index of the leftmost item with key greater than or equal to k.
        Return high + 1 if no such item qualifies.
        That is, j will be returned such that:
            all items of slice table[low:j] have key < k
            all items of slice table[j:high+1] have key >= k
        """
        if high < low:
            # no element qualifies
            return high + 1
        else:
            mid = (low + high) // 2
            if k == self._table[mid]._key:
                # found exact match
                return mid
            elif k < self._table[mid]._key:
                # note: may return mid
                return self._find_index(k, low, mid - 1)
            else:
                # answer is right of mid
                return self._find_index(k, mid + 1, high)

    # public behaviors
    def __init__(self):
        """
        Make an empty map.
        """
        self._table = []

    def __len__(self):
        """
        Return number of items in the map.
        """
        return len(self._table)

    def __getitem__(self, k):
        """
        Return value associated with key k (raise KeyError if not found).
        """
        j = self._find_index(k, 0, len(self._table) - 1)
        if j == len(self._table) or self._table[j]._key != k:
            raise KeyError('Key Error:' + repr(k))
        return self._table[j]._value

    def __setitem__(self, k, v):
        """
        Assign value v to key k, overwriting existing value if present.
        """
        j = self._find_index(k, 0, len(self._table) - 1)
        if j < len(self._table) and self._table[j]._key == k:
            # reassign value
            self._table[j]._value = v
        else:
            # adds new item
            self._table.insert(j, self._Item(k, v))

    def __delitem__(self, k):
        """
        Remove item associated with key k (raise KeyError if not found).
        """
        j = self._find_index(k, 0, len(self._table) - 1)
        if j == len(self._table) or self._table[j]._key != k:
            raise KeyError('Key Error: ' + repr(k))
        # delete item
        self._table.pop(j)

    def __iter__(self):
        """
        Generate keys of the map ordered from minimum to maximum.
        """
        for item in self._table:
            yield item._key

    def __reversed__(self):
        """
        Generate keys of the map ordered from maximum to minimum.
        """
        for item in reversed(self._table):
            yield item._key

    def find_min(self):
        """
        Return (key, value) pair with minimum key (or None if empty).
        """
        if len(self._table) > 0:
            return (self._table[0]._key, self._table[0]._value)
        else:
            return None

    def find_max(self):
        """
        Return (key, value) pair with maximum key (or None if empty).
        """
        if len(self._table) > 0:
            return (self._table[-1]._key, self._table[-1]._value)
        else:
            return None

    def find_ge(self, k):
        """
        Return (key, value) pair with least key greater than or equal to k.
        """
        # j's key >= k
        j = self._find_index(k, 0, len(self._table) - 1)
        if j < len(self._table):
            return (self._table[j]._key, self._table[j]._value)
        else:
            return None

    def find_lt(self, k):
        """
        Return (key, value) pair with greatest key strictly less than k.
        """
        # j's key >= k
        j = self._find_index(k, 0, len(self._table) - 1)
        if j > 0:
            # note use of j-1
            return (self._table[j - 1]._key, self._table[j - 1]._value)
        else:
            return None

    def find_gt(self, k):
        """
        Return (key, value) pair with least key strictly greater than k.
        """
        # j's key >= k
        j = self._find_index(k, 0, len(self._table) - 1)
        if j < len(self._table) and self._table[j]._key == k:
            # advanced past match
            j += 1
        if j < len(self._table):
            return (self._table[j]._key, self._table[j]._value)
        else:
            return None

    def find_range(self, start, stop):
        """
        Iterate all (key, value) pairs such that start <= key <= stop.
        If start is None, iteration begins with minimum key of map.
        If stop is None, iteration continues through the maximum key of map.
        """
        if start is None:
            j = 0
        else:
            # find first result
            j = self._find_index(start, 0, len(self._table) - 1)
        while j < len(self._table) and (stop is None or self._table[j]._key < stop):
            yield (self._table[j]._key, self._table[j]._value)
            j += 1

# so the code that is given to is is the recursive approach to finding the index

# In the original version of the _find_index method, when the key at mid is less than
#  k, it returns mid + 1. This means it effectively skips the key at the mid index and
#  continues the search in the upper half of the range. This behavior ensures that if
#  there are duplicate keys in the SortedTableMap, it always returns the index of the 
# first occurrence of the key.

# In contrast, the provided variant of _find_index recursively calls itself with a new 
# range that includes both the upper and lower halves when the key at mid is less than k.
#  This means that it might not return the index of the first occurrence of k but rather
#  the index of the last occurrence of k.

# To illustrate, consider a SortedTableMap with the following keys in sorted order:

test = [1, 2, 2, 3, 4, 5]
# If you use the original _find_index method to find the index of key 2, it
#  will correctly return the index 1 (the first occurrence). However, the
#  provided variant will return the index 2 (the last occurrence) because 
# it continues searching in the upper half of the range when it 
# encounters a key less than k.

# So, the two versions do not always produce the same result, and the
#  behavior of the provided variant may not align with the expected behavior 
# for finding the index of the first occurrence of a key in a SortedTableMap.

In [ ]:
# R-10.22 

# What is the expected running time of the methods for maintaining a maxima
#  set if we insert n pairs such that each pair has lower cost and performance
#  than one before it? 
# 
# What is contained in the sorted map at the end of this series of operations? 
# 
# What if each pair had a lower cost and higher performance than the one before it?

# what is a maxima set?

# Life is full of trade-offs. We often have to trade off a desired performance measure
# against a corresponding cost. Suppose, for the sake of an example, we are interested
# in maintaining a database rating automobiles by their maximum speeds and their
# cost. We would like to allow someone with a certain amount of money to query our
# database to find the fastest car they can possibly afford.

# We can model such a trade-off problem as this by using a key-value pair to
# model the two parameters that we are trading off, which in this case would be the
# pair (cost, speed) for each car.

# here is an example.

# the pairs that we will insert are: (100, 10) - (92 - 9) - (91 - 8) - (73 - 7)

# Unfortunately, if we implement this problem with using the SortedTableMap, the add behavior
# has O(n) worst-case running time.

# the result would be o(n^2) because for each pair we need to check the tradeoff and 
# insert the pair to a specific location



# FOR question R10-23

![a skip list](img/fig1010.png)

First Figure of A Skip List

---

![flags](img/fig1012.png)

Insertion in a Skip List

---

![deletion](img/fig1013.png)

Deletion in a Skip List

---

In [ ]:
# R-10.23 
# Draw an example skip list S that results from performing the following
# series of operations on the skip list shown in Figure 10.13: 
# del S[38], S[48] = x , S[24] = y , del S[55]. 
# 
# Record your coin flips, as well.

"""First of all, what are Skip Lists?"""

# In Section 10.3.1, we saw that a sorted array will allow O(log n)-time searches via the
# binary search algorithm. Unfortunately, update operations on a sorted array have
# O(n) worst-case running time because of the need to shift elements. In Chapter 7
# we demonstrated that linked lists support very efficient update operations, as long
# as the position within the list is identified. 
# 
# Unfortunately, we cannot perform fast
# searches on a standard linked list; for example, the binary search algorithm requires
# an efficient means for direct accessing an element of a sequence by index.
# 
# Skip lists provide a clever compromise to efficiently support search and update
# operations. A skip list S for a map M consists of a series of lists {S0, S1 , . . . , Sh}.
# Each list Si stores a subset of the items of M sorted by increasing keys, plus items
# with two sentinel keys denoted −∞and +∞, where −∞is smaller than every
# possible key that can be inserted in M and +∞is larger than every possible key
# that can be inserted in M. In addition, the lists in S satisfy the following:

# •List S0 contains every item of the map M (plus sentinels −∞and +∞).

# •For i = 1, . . . , h−1, list Si contains (in addition to −∞and +∞) a randomly
# generated subset of the items in list Si−1.

# •List Sh contains only −∞and +∞.

# An example of a skip list is shown in Figure 10.10. It is customary to visualize a
# skip list S with list S0 at the bottom and lists S1, . . . , Sh above it. Also, we refer to h
# as the height of skip list S.

# Intuitively, the lists are set up so that Si+1 contains more or less alternate items
# of Si. As we shall see in the details of the insertion method, the items in Si+1 are
# chosen at random from the items in Si by picking each item from Si to also be in
# Si+1 with probability 1/2. 
# 
# That is, in essence, we “flip a coin” for each item in S
# and place that item in Si+1 if the coin comes up “heads.” Thus, we expect S1 to have
# 1about n/2 items, S2 to have about n/4 items, and, in general, Si to have about n/2i
# 1items. In other words, we expect the height h of S to be about log n. 

# back to the question

# Creating a skip list example with the given elements and performing the requested operations
#  while recording coin flips:

# Initial skip list after operations on Fig10.13:

# Head -> 12 -> 17 -> 20 -> 31 -> 38 -> 39 -> 42 -> 44 -> 50 -> 55 -> Tail

# Now, let's perform the given operations:

# Delete S[38]:
# Coin Flips: HTTTTTTTTT
# After deleting 38:
# Head -> 12 -> 17 -> 20 -> 31 -> 39 -> 42 -> 44 -> 50 -> 55 -> Tail

# Set S[48] = x:
# Coin Flips: HHTT
# Since 48 is not in the original skip list, we insert it as follows:
# Head -> 12 -> 17 -> 20 -> 31 -> 39 -> 42 -> 44 -> 48(x) -> 50 -> 55 -> Tail

# Set S[24] = y:
# Coin Flips: HHTT
# Since 24 is not in the original skip list, we insert it as follows:
# Head -> 12 -> 17 -> 20 -> 24(y) -> 31 -> 39 -> 42 -> 44 -> 48(x) -> 50 -> 55 -> Tail

# Delete S[55]:
# Coin Flips: HHTT
# After deleting 55:
# Head -> 12 -> 17 -> 20 -> 24(y) -> 31 -> 39 -> 42 -> 44 -> 48(x) -> 50 -> Tail


In [ ]:
# R-10.24 
# Give a pseudo-code description of the delitem map operation when
# using a skip list.

# Procedure __delitem__(k):
# Input: A key k to delete from the skip list
# Output: None (key k is removed from the skip list)
# 
# 1. Let current be the top-left sentinel node of the skip list.
# 
# 2. Repeat until we reach the bottom level:
#    a. While the key of the next node is less than k, move current to the right.
#    b. If the key of the next node equals k, delete the node:
#       i. Remove the node's reference(s) at the current level(s).
#       ii. Move current to the right at the current level.
# 
# 3. If key k was found and deleted, repeat the process starting from the next 
# level (above the current level) until reaching the top level.
# 
# 4. Return, as key k has been successfully removed from the skip list.

In [19]:
# R-10.25 
# Give a concrete implementation of the pop method, in the context of a
# MutableSet abstract base class, that relies only on the five core set 
# behaviors described in Section 10.5.2.

"""Before we get to that, here are some definitions"""

# •A set is an unordered collection of elements, without duplicates, that typically
#  supports efficient membership tests. In essence, elements of a set are
# like keys of a map, but without any auxiliary values.
# 
# •A multiset (also known as a bag) is a set-like container that allows duplicates.
# 
# •A multimap is similar to a traditional map, in that it associates values with
# keys; however, in a multimap the same key can be mapped to multiple values.
#  For example, the index of this book maps a given term to one or more
# locations at which the term occurs elsewhere in the book.

"""Back to the question"""

# FRom book

# To aid in the creation of user-defined set classes, Python’s collections module provides
#  a MutableSet abstract base class (just as it provides the MutableMapping abstract
#  base class discussed in Section 10.1.3). The MutableSet base class provides
# concrete implementations for all methods described in Section 10.5.1, except for
# five core behaviors (add, discard, __contains__ , __len__ , and __iter__ ) that must
# be implemented by any concrete subclass.

from collections.abc import MutableSet

class MySet(MutableSet):
    def __init__(self):
        self.elements = set()

    def add(self, elem):
        self.elements.add(elem)

    def discard(self, elem):
        self.elements.discard(elem)

    def __contains__(self, elem):
        return elem in self.elements

    def __len__(self):
        return len(self.elements)

    def __iter__(self):
        return iter(self.elements)

    def pop(self):
        """We can only use the 5 methods we have."""
        if not self.elements:
            raise KeyError("pop from an empty set")
        elem_to_pop = next(iter(self.elements))  # Get an arbitrary element
        # set methods for insertion and deletion are - add and remove
        self.elements.remove(elem_to_pop)
        return elem_to_pop

# Example usage:
s = MySet()
s.add(1)
s.add(2)
s.add(3)

print(s.pop())  # This will remove and return an arbitrary element


1


In [24]:
# R-10.26 
# Give a concrete implementation of the isdisjoint method in the context
# of the MutableSet abstract base class, relying only on the five primary
# abstract methods of that class. Your algorithm should run in O(min(n, m))
# where n and m denote the respective cardinalities of the two sets.

from collections.abc import MutableSet

class MySet(MutableSet):
    def __init__(self):
        self.elements = set()

    def add(self, elem):
        self.elements.add(elem)

    def discard(self, elem):
        self.elements.discard(elem)

    def __contains__(self, elem):
        return elem in self.elements

    def __len__(self):
        return len(self.elements)

    def __iter__(self):
        return iter(self.elements)

    def isdisjoint(self, other):
        """Return True if the two sets are disjoint"""
        if type(other) != MySet:
            raise TypeError("Cannot make jointness decisions between these objects")
        # find the big set and the small one.
        if len(self) < len(other):
            for elem in self:
                if elem in other:
                    return False
            return True
        else:
            for elem in other:
                if elem in self:
                    return False
            return True

set1 = MySet()

set1.add(1)
set1.add(2)
set1.add(3)

set2 = MySet()

set2.add(4)
set2.add(5)

print(f"This should be True: {set1.isdisjoint(set2)}")

set2.add(1)
print(f"This should be False: {set1.isdisjoint(set2)}")

This should be True: True
This should be False: False


In [ ]:
"""Important. Skip List Basics."""

# R-10.27 
# What abstraction would you use to manage a database of friends’ birth-
# days in order to support efficient queries such as “find all friends whose
# birthday is today” and “find the friend who will be the next to celebrate a
# birthday”?

"""Answer"""

# Skip lists can be a good choice for managing a database of friends' birthdays when you
#  need to support efficient queries such as "find all friends whose birthday is today" and
#  "find the friend who will be the next to celebrate a birthday." Here's why skip lists
#  are a suitable data structure for this scenario:

# Efficient Searching: Skip lists are designed to provide efficient searching for elements. 
# They allow for logarithmic time complexity for search operations, making it quick to find 
# friends with specific birthdates.

# Ordered Data: Skip lists maintain data in a sorted order. Since birthdays are typically
#  sorted chronologically, this data structure naturally fits the requirements of this problem.

# Balanced Structure: Skip lists maintain a balanced structure, which ensures that operations
#  like searching for a friend's birthday or finding the next birthday celebration are 
# consistently efficient, even as the database grows.

# Insertions and Deletions: Skip lists also support efficient insertions and deletions.
#  If you need to add new friends to the database or remove friends, these operations can 
# be performed with logarithmic time complexity.

# Queries with Ranges: Skip lists can easily be adapted to handle more complex queries, such
#  as finding friends with birthdays in a specific date range. This is valuable if you want 
# to extend the functionality of your database.

# Easy to Implement: While skip lists are not as widely known as some other data structures
#  like binary search trees, they are relatively straightforward to implement compared to 
# more complex structures like balanced BSTs.

# Here's a high-level overview of how skip lists work:

# Skip lists consist of multiple layers or levels, with each level containing a sorted subset 
# of the elements from the lower level.

# At the top level, you have a sparse representation of the data, with only a few elements. 
# As you move down the levels, you have increasingly more elements.

# Each level allows for faster navigation through the list, effectively "skipping" over 
# large portions of the data, hence the name "skip list."

# When you want to find a specific birthday or perform a range query, you can start at the 
# top level and move down, reducing the search space at each level until you find the 
# desired element(s) or range.

# In summary, skip lists strike a good balance between simplicity of implementation and
#  efficient querying, making them a suitable choice for managing a database of 
# friends' birthdays, especially when you need to support various queries efficiently.

In [ ]:
# C-10.28 
# On page 406 of Section 10.1.3, we give an implementation of the method
# setdefault as it might appear in the MutableMapping abstract base class.
# While that method accomplishes the goal in a general fashion, its efficiency is less
#  than ideal. 

# here is the method
def setdefault(self, k ,d):
    try:
        return self[k]
    except KeyError:
        self[k] = d
        return d

# In particular, when the key is new, there will be
# a failed search due to the initial use of __getitem__ , and then a subsequent
#  insertion via __setitem__ . 
# 
# For a concrete implementation, such as the UnsortedTableMap, this is twice the
#  work because a complete scan of the table will take place during the failed __getitem__
#  , and  then another complete scan of the table takes place due to the implementation of
# __setitem__. 
# 
# A better solution is for the UnsortedTableMap class to override setdefault to
#  provide a direct solution that performs a single search.

# Give such an implementation of UnsortedTableMap.setdefault.

"""Answer"""

# To improve the efficiency of the setdefault method in the UnsortedTableMap class, we can
#  override it to directly search for the key and perform the insertion if the key is not
#  found, eliminating the need for two separate searches. Here's an implementation of 
# setdefault for UnsortedTableMap:

from collections.abc import MutableMapping

class UnsortedTableMap(MutableMapping):
    def __init__(self):
        self._table = []

    def __getitem__(self, k):
        for key, value in self._table:
            if key == k:
                return value
        raise KeyError(f'Key not found: {k}')

    def __setitem__(self, k, v):
        for i, (key, value) in enumerate(self._table):
            if key == k:
                self._table[i] = (k, v)
                return
        self._table.append((k, v))

    def __delitem__(self, k):
        for i, (key, value) in enumerate(self._table):
            if key == k:
                del self._table[i]
                return
        raise KeyError(f'Key not found: {k}')

    def __iter__(self):
        for key, value in self._table:
            yield key

    def __len__(self):
        return len(self._table)

    def setdefault(self, k, d):
        for key, value in self._table:
            if key == k:
                return value
        self._table.append((k, d))
        return d

# In this implementation of setdefault, we iterate through the _table list to
#  search for the key k. If we find it, we return the associated value. If the
#  key is not found, we directly append a new key-value pair to the _table list
#  and return the default value d. This approach avoids the need for two separate
#  searches, making it more efficient.

In [29]:
# C-10.29 
# Repeat Exercise C-10.28 for the ProbeHashMap class

from collections.abc import MutableMapping
from random import randrange

class MapBase(MutableMapping):
    """Our own abstract base class that includes a non public _Item class"""

    # nested _Item class
    class _Item:
        __slots__ = "_key", "_value"
        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key      # only compare keys

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
             return self._key < other._key      # compare based on keys

class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression"""
    def __init__(self, cap = 11, p = 109345121):
        """Make an empty hash-table map"""
        self._table = cap * [None]
        self._n = 0                         # Number of entries in the map
        self._prime = p                     # prime for MAD compression
        self._scale = 1 + randrange(p-1)    # scale from 1 to p-1 for MAD
        self._shift = randrange(p)          # shift from 0 to p-1 for MAD

    def _hash_function(self,k):
        return (hash(k) *  self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self,k):
        j = self._hash_function(k)
        return self._bucket_getitem(j,k)

    def __setitem__(self, k, v) :
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)               # subroutine maintains self._n
        if self._n > len(self._table) // 2:         # keep load factor <= 0.5
            self._resize(2*len(self._table) - 1)    # number 2^x - 1 often prime

    def _resize(self, c):               # resize bucket array to capacity c
        old = list(self.items())        # use iteration to record existing items
        self._table = c * [None]        # reset the table to desired capacity
        self._n = 0                     # n recomputed during subsequent adds
        for (k,v) in old:       
            self[k] = v                  # reinsert old key value pair

    def __delitem__(self, k ):
        j = self._hash_function(k)
        self._bucket_delitem(j,k)       # may Raise KeyError
        self._n -= 1

class ProbeHashMap(HashMapBase):
    """Hash map implemented with linear probing for collision resolution."""
    _AVAIL = object() # sentinel object that marks locations of previous deletions

    def _is_avaliable(self, j):
        """Return True if index j is avaliable in table."""
        return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL

    def _find_slot(self, j ,k):
        r"""
        Search for key k in bucket at index j.
        Return (success, index) tuple, described as follows:
        If match was found, success is True and index denotes its location.
        If no match found, success is False and index denotes first available slot.
        """
        firstAvail = None
        while True:
            if self._is_avaliable(j):
                if firstAvail is None:
                    firstAvail = j                          # mark this as first avail
                if self._table[j] is None:
                    return (False, firstAvail)              # search has failed
            elif k == self._table[j]._key:
                return (True, j)                            # found a match
            j = (j + 1) % len(self._table)                  # keep looking (cyclically)

    def bucket_getitem(self, j, k): 
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError( "Key Error:" + repr(k))         # no match found
        return self._table[s]._value

    def bucket_setitem(self, j, k, v): 
        found, s = self._find_slot(j, k)
        if not found:
            self._table[s] = self._Item(k,v)                # insert new item
            self._n += 1                                    # size has increased
        else:
            self._table[s]._value = v                       # overwrite existing

    def bucket_delitem(self, j, k): 
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError( "Key Error:" + repr(k))         # no match found
        self._table[s] = ProbeHashMap._AVAIL                # mark as vacated

    def __iter__(self):
        for j in range(len(self. table)):                   # scan entire table
            if not self._is_avaliable(j):
                yield self._table[j]._key

    def setdefault(self, k, d):
        j = self._hash_function(k)  # Compute the initial bucket index
        found, s = self._find_slot(j, k)  # Attempt to find the key in the bucket

        if found:
            return self._table[s]._value  # If found, return the existing value

        # Key not found, insert the new key-value pair
        self._table[s] = self._Item(k, d)
        self._n += 1

        if self._n > len(self._table) // 2:
            self._resize(2 * len(self._table) - 1)

        return d  # Return the default value, as it has been inserted

phm = ProbeHashMap()

phm.bucket_setitem(0 , "a", 1)
phm.bucket_setitem(1 , "b", 2)
phm.bucket_setitem(2 , "c", 3)

phm.bucket_getitem(1, "b")

"""In this setdefault implementation:"""

# - We first calculate the initial bucket index (j) using the hash function.
# - Then, we attempt to find the key in the bucket using the _find_slot method.
# - If the key is found (found is True), we return the existing value associated with the key.
# - If the key is not found, we insert the new key-value pair into the table.
# - If the load factor exceeds 0.5, we resize the table to maintain efficiency.
# - Finally, we return the default value d, whether it's newly inserted or already existed.

# This implementation ensures that the setdefault method efficiently finds and 
# inserts key-value pairs in the ProbeHashMap without unnecessary searches or insertions.

2

In [30]:
# C-10.30 
# Repeat Exercise C-10.28 for the ChainHashMap class.

# Here is ChainHashMap

from collections.abc import MutableMapping
from random import randrange

class MapBase(MutableMapping):
    """Our own abstract base class that includes a non public _Item class"""

    # nested _Item class
    class _Item:
        __slots__ = "_key", "_value"
        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key      # only compare keys

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
             return self._key < other._key      # compare based on keys

class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression"""
    def __init__(self, cap = 11, p = 109345121):
        """Make an empty hash-table map"""
        self._table = cap * [None]
        self._n = 0                         # Number of entries in the map
        self._prime = p                     # prime for MAD compression
        self._scale = 1 + randrange(p-1)    # scale from 1 to p-1 for MAD
        self._shift = randrange(p)          # shift from 0 to p-1 for MAD

    def _hash_function(self,k):
        return (hash(k) *  self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self,k):
        j = self._hash_function(k)
        return self._bucket_getitem(j,k)

    def __setitem__(self, k, v) :
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)               # subroutine maintains self._n
        if self._n > len(self._table) // 2:         # keep load factor <= 0.5
            self._resize(2*len(self._table) - 1)    # number 2^x - 1 often prime

    def _resize(self, c):               # resize bucket array to capacity c
        old = list(self.items())        # use iteration to record existing items
        self._table = c * [None]        # reset the table to desired capacity
        self._n = 0                     # n recomputed during subsequent adds
        for (k,v) in old:       
            self[k] = v                  # reinsert old key value pair

    def __delitem__(self, k ):
        j = self._hash_function(k)
        self._bucket_delitem(j,k)       # may Raise KeyError
        self._n -= 1

class ChainHashMap(HashMapBase):
    """Hash map implemented with separate chaining for collision resolution."""

    def _bucket_getitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))  # no match found
        return bucket[k]  # may raise KeyError

    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap()  # bucket is new to the table
        oldsize = len(self._table[j])
        self._table[j][k] = v
        if len(self._table[j]) > oldsize:  # key was new to the table
            self._n += 1  # increase overall map size

    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))  # no match found
        del bucket[k]  # may raise KeyError

    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:  # a nonempty slot
                for key in bucket:
                    yield key

    # HERE IS THE ADDED SET DEFAULT METHOD
    def setdefault(self, k, d):
        j = self._hash_function(k)  # Compute the initial bucket index
        bucket = self._table[j]  # Get the bucket at the initial index

        if bucket is None:
            # If the bucket is empty, Make a new UnsortedTableMap
            bucket = UnsortedTableMap()
            self._table[j] = bucket

        if k not in bucket:
            # If the key is not in the bucket, insert the new key-value pair
            bucket[k] = d
            self._n += 1

            if self._n > len(self._table) // 2:
                self._resize(2 * len(self._table) - 1)  # Resize if load factor exceeds 0.5

        return bucket.get(k, d)  # Return the value associated with the key   


# In this setdefault implementation:

# - We first calculate the initial bucket index (j) using the hash function.
# - Then, we retrieve the bucket at the initial index.
# - If the bucket is empty (i.e., None), we Make a new UnsortedTableMap to use as the bucket.
# - We check if the key is in the bucket. If not, we insert the new key-value pair into the bucket.
# - If the load factor exceeds 0.5, we resize the table to maintain efficiency.
# - Finally, we return the value associated with the key, either the existing one or the
#  default value d if it's a new insertion.     

In [32]:
# C-10.31 
# For an ideal compression function, the capacity of the bucket array for a
# hash table should be a prime number. Therefore, we consider the problem
# of locating a prime number in a range [M, 2M]. 
# 
# Implement a method for finding such a prime by using the sieve algorithm. 
# 
# In this algorithm, we allocate a 2M cell Boolean array A, such that cell i is
#  associated with the integer i. We then initialize the array cells to
#  all be “true” and we “mark off” all the cells that are multiples 
# of 2, 3, 5, 7, and so on. 
# 
# This process  can stop after it reaches a number larger than √2M. (Hint: Consider a
# bootstrapping method for finding the primes up to √2M.

import math

def primes(size):
    seq = [True] * size
    seq[0] = seq[1] = False  # 0 and 1 are not prime

    for index in range(2, int(math.sqrt(size)) + 1):
        if seq[index]:
            # Mark multiples of index as non-prime
            for multiple in range(index * index, size, index):
                seq[multiple] = False

    return seq

def is_prime(x):
    """Return True if the given integer number is prime"""
    if x < 2:
        return False
    for elem in range(2, int(math.sqrt(x)) + 1):
        if x % elem == 0:
            return False
    return True

# Example usage:
M = 10
size = 2 * M  # Replace M with your desired value
prime_flags = primes(size)

# Find the first prime in the range [M, 2M]
for i in range(M, 2 * M + 1):
    if prime_flags[i]:
        print(f"The first prime number in the range [{M}, {2 * M}] is {i}.")
        break
else:
    print(f"No prime number found in the range [{M}, {2 * M}].")

The first prime number in the range [10, 20] is 11.


In [ ]:
# C-10.32 
# Perform experiments on our ChainHashMap and ProbeHashMap classes
# to measure its efficiency using random key sets and varying limits on the
# load factor (see Exercise R-10.15)

# To perform experiments on the ChainHashMap and ProbeHashMap classes to measure
#  their efficiency with varying load factors, you can follow these steps:

# Make test cases with random key sets.

# Insert keys and values into both ChainHashMap and ProbeHashMap instances with different load factors.

# Measure the time taken for insertion, retrieval, and deletion operations.

# Analyze the results to see how performance varies with different load factors.


from random import randrange
import random
import time
from collections.abc import MutableMapping

class UnsortedTableMap(MutableMapping):
    def __init__(self):
        self._table = []

    def __getitem__(self, k):
        for key, value in self._table:
            if key == k:
                return value
        raise KeyError(f'Key not found: {k}')

    def __setitem__(self, k, v):
        for i, (key, value) in enumerate(self._table):
            if key == k:
                self._table[i] = (k, v)
                return
        self._table.append((k, v))

    def __delitem__(self, k):
        for i, (key, value) in enumerate(self._table):
            if key == k:
                del self._table[i]
                return
        raise KeyError(f'Key not found: {k}')

    def __iter__(self):
        for key, value in self._table:
            yield key

    def __len__(self):
        return len(self._table)

    def setdefault(self, k, d):
        for key, value in self._table:
            if key == k:
                return value
        self._table.append((k, d))
        return d

class MapBase(MutableMapping):
    """Our own abstract base class that includes a non public _Item class"""

    # nested _Item class
    class _Item:
        __slots__ = "_key", "_value"
        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key      # only compare keys

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
             return self._key < other._key      # compare based on keys

class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression"""
    def __init__(self, cap = 11, p = 109345121):
        """Make an empty hash-table map"""
        self._table = cap * [None]
        self._n = 0                         # Number of entries in the map
        self._prime = p                     # prime for MAD compression
        self._scale = 1 + randrange(p-1)    # scale from 1 to p-1 for MAD
        self._shift = randrange(p)          # shift from 0 to p-1 for MAD

    def _hash_function(self,k):
        return (hash(k) *  self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self,k):
        j = self._hash_function(k)
        return self._bucket_getitem(j,k)

    def __setitem__(self, k, v) :
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)               # subroutine maintains self._n
        if self._n > len(self._table) // 2:         # keep load factor <= 0.5
            self._resize(2*len(self._table) - 1)    # number 2^x - 1 often prime

    def _resize(self, c):               # resize bucket array to capacity c
        old = list(self.items())        # use iteration to record existing items
        self._table = c * [None]        # reset the table to desired capacity
        self._n = 0                     # n recomputed during subsequent adds
        for (k,v) in old:       
            self[k] = v                  # reinsert old key value pair

    def __delitem__(self, k ):
        j = self._hash_function(k)
        self._bucket_delitem(j,k)       # may Raise KeyError
        self._n -= 1

class ProbeHashMap(HashMapBase):
    """Hash map implemented with linear probing for collision resolution."""
    _AVAIL = object() # sentinel object that marks locations of previous deletions

    def _is_avaliable(self, j):
        """Return True if index j is avaliable in table."""
        return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL

    def _find_slot(self, j ,k):
        r"""
        Search for key k in bucket at index j.
        Return (success, index) tuple, described as follows:
        If match was found, success is True and index denotes its location.
        If no match found, success is False and index denotes first available slot.
        """
        firstAvail = None
        while True:
            if self._is_avaliable(j):
                if firstAvail is None:
                    firstAvail = j                          # mark this as first avail
                if self._table[j] is None:
                    return (False, firstAvail)              # search has failed
            elif k == self._table[j]._key:
                return (True, j)                            # found a match
            j = (j + 1) % len(self._table)                  # keep looking (cyclically)

    def bucket_getitem(self, j, k): 
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError( "Key Error:" + repr(k))         # no match found
        return self._table[s]._value

    def bucket_setitem(self, j, k, v): 
        found, s = self._find_slot(j, k)
        if not found:
            self._table[s] = self._Item(k,v)                # insert new item
            self._n += 1                                    # size has increased
        else:
            self._table[s]._value = v                       # overwrite existing

    def bucket_delitem(self, j, k): 
        found, s = self._find_slot(j, k)
        if not found:
            raise KeyError( "Key Error:" + repr(k))         # no match found
        self._table[s] = ProbeHashMap._AVAIL                # mark as vacated

    def __iter__(self):
        for j in range(len(self. table)):                   # scan entire table
            if not self._is_avaliable(j):
                yield self._table[j]._key

    def setdefault(self, k, d):
        j = self._hash_function(k)  # Compute the initial bucket index
        found, s = self._find_slot(j, k)  # Attempt to find the key in the bucket

        if found:
            return self._table[s]._value  # If found, return the existing value

        # Key not found, insert the new key-value pair
        self._table[s] = self._Item(k, d)
        self._n += 1

        if self._n > len(self._table) // 2:
            self._resize(2 * len(self._table) - 1)

        return d  # Return the default value, as it has been inserted

class ChainHashMap(HashMapBase):
    """Hash map implemented with separate chaining for collision resolution."""

    def _bucket_getitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))  # no match found
        return bucket[k]  # may raise KeyError

    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap()  # bucket is new to the table
        oldsize = len(self._table[j])
        self._table[j][k] = v
        if len(self._table[j]) > oldsize:  # key was new to the table
            self._n += 1  # increase overall map size

    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))  # no match found
        del bucket[k]  # may raise KeyError

    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:  # a nonempty slot
                for key in bucket:
                    yield key

    # HERE IS THE ADDED SET DEFAULT METHOD
    def setdefault(self, k, d):
        j = self._hash_function(k)  # Compute the initial bucket index
        bucket = self._table[j]  # Get the bucket at the initial index

        if bucket is None:
            # If the bucket is empty, Make a new UnsortedTableMap
            bucket = UnsortedTableMap()
            self._table[j] = bucket

        if k not in bucket:
            # If the key is not in the bucket, insert the new key-value pair
            bucket[k] = d
            self._n += 1

            if self._n > len(self._table) // 2:
                self._resize(2 * len(self._table) - 1)  # Resize if load factor exceeds 0.5

        return bucket.get(k, d)  # Return the value associated with the key   

# Function to generate a random key set
def generate_random_key_set(size):
    return [random.randint(1, 10000) for _ in range(size)]

# Function to measure the performance of a hash map
def measure_performance(hash_map, keys, load_factor):
    start_time = time.time()

    # Insert keys and values
    for key in keys:
        hash_map[key] = key

    insertion_time = time.time() - start_time

    # Measure retrieval time
    start_time = time.time()
    for key in keys:
        _ = hash_map[key]

    retrieval_time = time.time() - start_time

    # Measure deletion time
    start_time = time.time()
    for key in keys:
        del hash_map[key]

    deletion_time = time.time() - start_time

    return {
        "Load Factor": load_factor,
        "Insertion Time": insertion_time,
        "Retrieval Time": retrieval_time,
        "Deletion Time": deletion_time,
    }

# Main experiment loop
load_factors = [0.5, 0.6, 0.7, 0.8, 0.9]
key_set_size = 1000  # Adjust as needed
num_experiments = 1  # Number of experiments per load factor

for load_factor in load_factors:
    print(f"Load Factor: {load_factor}")
    for _ in range(num_experiments):
        keys = generate_random_key_set(int(key_set_size * load_factor))

        # Test ChainHashMap
        chain_map = ChainHashMap()
        chain_results = measure_performance(chain_map, keys, load_factor)

        # Test ProbeHashMap
        probe_map = ProbeHashMap()
        probe_results = measure_performance(probe_map, keys, load_factor)

        print(f"ChainHashMap: {chain_results}")
        print(f"ProbeHashMap: {probe_results}")
        print()

In [ ]:
# C-10.33 
# Our implementation of separate chaining in ChainHashMap conserves
# memory by representing empty buckets in the table as None, rather than
# as empty instances of a secondary structure. Because many of these 
# buckets will hold a single item, a better optimization is to have those slots of
# the table directly reference the Item instance, and to reserve use of secondary
#  containers for buckets that have two or more items. Modify our
# implementation to provide this additional optimization.

# Original Code

class MapBase(MutableMapping):
    """Our own abstract base class that includes a non public _Item class"""

    # nested _Item class
    class _Item:
        __slots__ = "_key", "_value"
        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key      # only compare keys

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
             return self._key < other._key      # compare based on keys

class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression"""
    def __init__(self, cap = 11, p = 109345121):
        """Make an empty hash-table map"""
        self._table = cap * [None]
        self._n = 0                         # Number of entries in the map
        self._prime = p                     # prime for MAD compression
        self._scale = 1 + randrange(p-1)    # scale from 1 to p-1 for MAD
        self._shift = randrange(p)          # shift from 0 to p-1 for MAD

    def _hash_function(self,k):
        return (hash(k) *  self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self,k):
        j = self._hash_function(k)
        return self._bucket_getitem(j,k)

    def __setitem__(self, k, v) :
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)               # subroutine maintains self._n
        if self._n > len(self._table) // 2:         # keep load factor <= 0.5
            self._resize(2*len(self._table) - 1)    # number 2^x - 1 often prime

    def _resize(self, c):               # resize bucket array to capacity c
        old = list(self.items())        # use iteration to record existing items
        self._table = c * [None]        # reset the table to desired capacity
        self._n = 0                     # n recomputed during subsequent adds
        for (k,v) in old:       
            self[k] = v                  # reinsert old key value pair

    def __delitem__(self, k ):
        j = self._hash_function(k)
        self._bucket_delitem(j,k)       # may Raise KeyError
        self._n -= 1

class ChainHashMap(HashMapBase):
    """Hash map implemented with separate chaining for collision resolution."""

    def _bucket_getitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))  # no match found
        return bucket[k]  # may raise KeyError

    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = UnsortedTableMap()  # bucket is new to the table
        oldsize = len(self._table[j])
        self._table[j][k] = v
        if len(self._table[j]) > oldsize:  # key was new to the table
            self._n += 1  # increase overall map size

    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        if bucket is None:
            raise KeyError('Key Error: ' + repr(k))  # no match found
        del bucket[k]  # may raise KeyError

    def __iter__(self):
        for bucket in self._table:
            if bucket is not None:  # a nonempty slot
                for key in bucket:
                    yield key

    # HERE IS THE ADDED SET DEFAULT METHOD
    def setdefault(self, k, d):
        j = self._hash_function(k)  # Compute the initial bucket index
        bucket = self._table[j]  # Get the bucket at the initial index

        if bucket is None:
            # If the bucket is empty, Make a new UnsortedTableMap
            bucket = UnsortedTableMap()
            self._table[j] = bucket

        if k not in bucket:
            # If the key is not in the bucket, insert the new key-value pair
            bucket[k] = d
            self._n += 1

            if self._n > len(self._table) // 2:
                self._resize(2 * len(self._table) - 1)  # Resize if load factor exceeds 0.5

        return bucket.get(k, d)  # Return the value associated with the key   

"""Answer"""

# To optimize the ChainHashMap implementation by directly referencing the Item instances 
# in slots that hold a single item and reserving secondary containers for buckets with
#  two or more items, you can make the following changes to the ChainHashMap class:
# 
# Make a new _Item class that stores a key-value pair.
# 
# Modify the _Item class to include a reference to the next item in the chain (a singly
#  linked list). This reference will be None for the last item in the chain.
# 
# Update the _Item class to include a count of the number of items in the chain. You can
#  increment this count each time you add an item to the chain.
# 
# Update the _bucket_setitem and _bucket_delitem methods to handle chaining items within 
# a slot. You will need to check if an item already exists in the slot. If it does, you can
#  add the new item to the end of the chain or remove the item from the chain.

class ChainHashMap_(HashMapBase):
    """Hash map implemented with separate chaining for collision resolution."""

    class _Item:
        __slots__ = "_key", "_value", "_next", "_count"
        def __init__(self, k, v):
            self._key = k
            self._value = v
            self._next = None  # Reference to the next item in the chain
            self._count = 1    # Count of items in the chain

    def _bucket_getitem(self, j, k):
        bucket = self._table[j]
        while bucket is not None:
            if bucket._key == k:
                return bucket._value
            bucket = bucket._next
        raise KeyError('Key Error: ' + repr(k))  # no match found

    def _bucket_setitem(self, j, k, v):
        if self._table[j] is None:
            self._table[j] = self._Item(k, v)
        else:
            bucket = self._table[j]
            while bucket:
                if bucket._key == k:
                    # Key already exists, update the value
                    bucket._value = v
                    return
                prev_bucket = bucket
                bucket = bucket._next
            # Key not found, add a new item to the chain
            prev_bucket._next = self._Item(k, v)
            self._table[j]._count += 1  # Increment the count

        self._n += 1  # Increase overall map size

    def _bucket_delitem(self, j, k):
        bucket = self._table[j]
        prev_bucket = None
        while bucket is not None:
            if bucket._key == k:
                if prev_bucket:
                    # Remove the item from the chain
                    prev_bucket._next = bucket._next
                else:
                    # The first item in the chain is removed
                    self._table[j] = bucket._next
                self._table[j]._count -= 1  # Decrement the count
                if self._table[j]._count == 1:
                    # If only one item left in the chain, remove the chain
                    self._table[j] = self._table[j]._next
                self._n -= 1  # Decrease overall map size
                return
            prev_bucket = bucket
            bucket = bucket._next
        raise KeyError('Key Error: ' + repr(k))  # no match found

In [10]:
"""Genius - Real Insight"""

# C-10.34 
# Computing a hash code can be expensive, especially for lengthy keys.
# 
#  In our hash table implementations, we compute the hash code when first inserting
#  an item, and recompute each item’s hash code each time we resize
# our table. 
# 
# Python’s dict class makes an interesting trade-off. The hash
# code is computed once, when an item is inserted, and the hash code is
# stored as an extra field of the item composite, so that it need not be recomputed.
#
#  Reimplement our HashTableBase class to use such an approach.

from collections.abc import MutableMapping

class HashTableBase(MutableMapping):
    """Abstract base class for hash table implementations."""

    def __init__(self, cap=11):
        """Make an empty hash table."""
        self._table = cap * [None]                      # Make an empty table (list)
        self._n = 0                                     # Number of entries in the map
        self._prime = 109345121                         # Prime number for MAD compression
        self._scale = 1 + randrange(self._prime - 1)    # Scale for MAD compression
        self._shift = randrange(self._prime)            # Shift for MAD compression
        self._hash_codes = {}                           # Dictionary to store hash codes

    def __len__(self):
        """Return the number of entries in the hash table."""
        return self._n

    def _hash_function(self, k):
        """Compute the hash code for key k."""
        if k in self._hash_codes:
            return self._hash_codes[k]  # Use the stored hash code
        hash_code = (hash(k) * self._scale + self._shift) % self._prime % len(self._table)
        self._hash_codes[k] = hash_code  # Store the hash code
        return hash_code

    def __getitem__(self, k):
        """Return the value associated with key k (raise KeyError if not found)."""
        j = self._hash_function(k)
        return self._bucket_getitem(j, k)

    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present."""
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)
        if self._n > len(self._table) // 2:
            self._resize(2 * len(self._table) - 1)

    def __delitem__(self, k):
        """Remove the item associated with key k (raise KeyError if not found)."""
        j = self._hash_function(k)
        self._bucket_delitem(j, k)
        self._n -= 1

    def __iter__(self):
        """Generate an iterator of keys in the hash table."""
        for bucket in self._table:
            if bucket is not None:
                for key in bucket:
                    yield key

# With this implementation, the hash code for each key is computed only once and stored
#  in the _hash_codes dictionary. Subsequent operations that require the hash code can 
# retrieve it from this dictionary, avoiding the need to recompute it. This optimization can
#  reduce the overhead of computing hash codes, especially for lengthy keys.

In [ ]:
# C-10.35 
# Describe how to perform a removal from a hash table that uses linear
# probing to resolve collisions where we do not use a special marker to
# represent deleted elements. That is, we must rearrange the contents so that
# it appears that the removed entry was never inserted in the first place.

# When removing an entry from a hash table that uses linear probing to resolve
#  collisions without using a special marker for deleted elements, you need to 
# ensure that the removed entry does not break the linear probing sequence and 
# that it appears as if it was never inserted in the first place. Here's a step-by-step
#  guide on how to perform such a removal:

# Locate the Entry: First, use the hash function to locate the index (slot) where the
#  entry you want to remove is stored. If the entry is not found at that index, follow
#  the linear probing sequence to search for it.

# Mark the Entry as Deleted: Once you find the entry you want to remove, you cannot 
# simply delete it, as it would break the linear probing sequence. Instead, you should 
# mark it as deleted, which effectively indicates that the slot is available for new
#  insertions but still preserves the sequence.

# Adjust the Linear Probing Sequence: After marking the entry as deleted, you need to
#  adjust the linear probing sequence to ensure that you can continue searching for 
# other entries correctly. To do this, you should continue probing linearly from the
#  marked slot, looking for the next valid entry in the sequence.

# Stop When a Valid Entry Is Found: Continue probing until you either find the next
#  valid entry (i.e., an entry that is not deleted) or reach an empty slot (indicating
#  the end of the sequence). When a valid entry is found, you can leave it where it is.

# Repeat as Necessary: If there are additional entries in the probing sequence, you can
#  repeat steps 2 through 4 as needed until you either find all the entries you need or
#  reach an empty slot.

# Update the Table Size: If the number of marked entries (deleted entries) becomes 
# significant compared to the total number of slots in the table, you may need to consider
#  resizing the table to maintain good performance. Resizing typically involves creating
#  a larger table and rehashing the valid entries into the new table while ignoring the
#  marked (deleted) entries.

# By following these steps, you can perform a removal from a hash table using linear probing
#  without the need for a special marker for deleted elements. This approach ensures that
#  the linear probing sequence remains intact and that the removed entry does not disrupt
#  the table's operations.

In [ ]:
# C-10.36 

# The quadratic probing strategy has a clustering problem related to the way
# it looks for open slots. Namely, when a collision occurs at bucket h(k), it 
# checks buckets A[(h(k) + i2) mod N], for i = 1, 2, . . . , N −1.
#   a. Show that i2 mod N will assume at most (N + 1)/2 distinct values,
# for N prime, as i ranges from 1 to N −1. As a part of this 
# justification, note that i2 mod N = ( N −i)2 mod N for all i.
#   b. A better strategy is to choose a prime N such that N mod 4 = 3 and 
# then to check the buckets A[(h(k) ±i2) mod N] as i ranges from 1
# to (N −1)/2, alternating between plus and minus. Show that this
# alternate version is guaranteed to check every bucket in A.

# a. To show that i^2 mod N will assume at most (N + 1)/2 distinct values for
#  N prime, as i ranges from 1 to N - 1, we can use the property 
# that i^2 mod N = (N - i)^2 mod N for all i. 
# 
# Let's consider the values of i^2 mod N for i ranging from 1 to N - 1:
# 
#   - When i = 1, i^2 mod N = 1^2 mod N = 1.
#   - When i = 2, i^2 mod N = 2^2 mod N = 4 mod N. But since N is prime, all values
#  from 1 to N - 1 are relatively prime to N, and N cannot divide 4. 
# Therefore, 4 mod N is distinct from 1.
#   - When i = 3, i^2 mod N = 3^2 mod N = 9 mod N. Similarly, since N is prime, all 
# values from 1 to N - 1 are relatively prime to N, and N cannot divide 9. 
# Therefore, 9 mod N is distinct from both 1 and 4 mod N.
#   - This pattern continues, and for each i, i^2 mod N is distinct from the 
# previously calculated values.

# As you can see, i^2 mod N takes on distinct values for each value of i from 1 
# to N - 1. Therefore, it assumes at most (N + 1)/2 distinct values.

# b. To show that the strategy of choosing a prime N such that N mod 4 = 3 and 
# checking the buckets A[(h(k) ± i^2) mod N] as i ranges from 1 to (N - 1)/2, alternating
#  between plus and minus, guarantees checking every bucket in A, we can consider
#  how this approach distributes the probing sequence:
# 
#   - When i = 1, it checks buckets A[(h(k) + 1^2) mod N] and A[(h(k) - 1^2) mod N].
#   - When i = 2, it checks buckets A[(h(k) + 2^2) mod N] and A[(h(k) - 2^2) mod N].
#   - This pattern continues, with each i, it checks two new buckets, one using the
#  plus sign and one using the minus sign, ensuring that all buckets are eventually checked.

# Since N is prime, this approach ensures that the probing sequence covers every 
# bucket in A because the values of i^2 mod N are distinct for each i from 1 
# to (N - 1)/2, and alternating between plus and minus ensures that both sides
#  of the collision point are explored. This approach helps reduce clustering
# issues associated with quadratic probing.


In [2]:
# C-10.37 
# Refactor our ProbeHashMap design so that the sequence of secondary
# probes for collision resolution can be more easily customized. Demonstrate
#  your new framework by providing separate concrete subclasses for
# linear probing and quadratic probing.

# To refactor the ProbeHashMap design so that the sequence of secondary probes
#  for collision resolution can be more easily customized, we can Make a
#  new class called CustomProbeHashMap that allows us to define our own probing 
# sequence strategy. This new class will provide a framework for defining
#  custom probing sequences.

from collections.abc import MutableMapping

class MapBase(MutableMapping):
    """Our own abstract base class that includes a non public _Item class"""

    # nested _Item class
    class _Item:
        __slots__ = "_key", "_value"
        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __eq__(self, other):
            return self._key == other._key      # only compare keys

        def __ne__(self, other):
            return not (self == other)

        def __lt__(self, other):
             return self._key < other._key      # compare based on keys

class HashMapBase(MapBase):
    """Abstract base class for map using hash-table with MAD compression"""
    def __init__(self, cap = 11, p = 109345121):
        """Make an empty hash-table map"""
        self._table = cap * [None]
        self._n = 0                         # Number of entries in the map
        self._prime = p                     # prime for MAD compression
        self._scale = 1 + randrange(p-1)    # scale from 1 to p-1 for MAD
        self._shift = randrange(p)          # shift from 0 to p-1 for MAD

    def _hash_function(self,k):
        return (hash(k) *  self._scale + self._shift) % self._prime % len(self._table)

    def __len__(self):
        return self._n

    def __getitem__(self,k):
        j = self._hash_function(k)
        return self._bucket_getitem(j,k)

    def __setitem__(self, k, v) :
        j = self._hash_function(k)
        self._bucket_setitem(j, k, v)               # subroutine maintains self._n
        if self._n > len(self._table) // 2:         # keep load factor <= 0.5
            self._resize(2*len(self._table) - 1)    # number 2^x - 1 often prime

    def _resize(self, c):               # resize bucket array to capacity c
        old = list(self.items())        # use iteration to record existing items
        self._table = c * [None]        # reset the table to desired capacity
        self._n = 0                     # n recomputed during subsequent adds
        for (k,v) in old:       
            self[k] = v                  # reinsert old key value pair

    def __delitem__(self, k ):
        j = self._hash_function(k)
        self._bucket_delitem(j,k)       # may Raise KeyError
        self._n -= 1

class CustomProbeHashMap(HashMapBase):
    def __init__(self, cap=11, p=109345121, probing_sequence=None):
        super().__init__(cap, p)
        self.probing_sequence = probing_sequence or self.default_probing_sequence

    def _get_probe_indices(self, j, k):
        for i in self.probing_sequence():
            yield (j + i) % len(self._table)

    def _find_slot(self, j, k):
        firstAvail = None
        for i in self._get_probe_indices(j, k):
            if self._is_available(i):
                if firstAvail is None:
                    firstAvail = i
                if self._table[i] is None:
                    return (False, firstAvail)
            elif k == self._table[i]._key:
                return (True, i)
        return (False, firstAvail)

    def default_probing_sequence(self):
        """Default probing sequence (linear probing)."""
        i = 1
        while True:
            yield i
            i += 1

    # You can add other custom probing sequences as needed

# Example usage:
# linear_probing_map = CustomProbeHashMap(probing_sequence=CustomProbeHashMap.default_probing_sequence)
# quadratic_probing_map = CustomProbeHashMap(probing_sequence=my_custom_quadratic_sequence)

# With this design, you can Make instances of CustomProbeHashMap and specify your
#  own probing sequence strategy as a function. The default_probing_sequence method 
# provides a linear probing sequence by default, but you can define your own custom 
# probing sequences and pass them as arguments when creating instances of CustomProbeHashMap.
#  This allows you to easily customize the collision resolution strategy for the hash map.

In [ ]:
# C-10.38 
# Design a variation of binary search for performing the multimap 
# operation find_all(k) implemented with a sorted search table that includes 
# duplicates, and show that it runs in time O(s+ log n), where n is the number
# of elements in the dictionary and s is the number of items with given key k.

# To perform the find_all(k) operation efficiently with a sorted search table 
# that includes duplicates, you can use a modified binary search algorithm. 
# Here's a high-level outline of how to achieve this and analyze its time complexity:

# Sort the search table by the keys.

#       Use binary search to find the first occurrence of key k in the sorted table.
#  Let's call this index first_occurrence.

#       Use binary search again to find the last occurrence of key k in the sorted table.
# Let's call this index last_occurrence.

#       All elements with key k are now in the range from first_occurrence to
#  last_occurrence in the sorted table.

def find_all(self, k):
    first_occurrence = self.find_first_occurrence(k)
    last_occurrence = self.find_last_occurrence(k)
    
    if first_occurrence is None or last_occurrence is None:
        return []  # Key not found
    
    # Extract elements with key k from the sorted search table
    result = []
    for i in range(first_occurrence, last_occurrence + 1):
        result.append(self.sorted_table[i])
    
    return result

# Now, let's analyze the time complexity:

# Sorting the table initially takes O(n * log n) time, where n is the
#  number of elements in the dictionary.

# Binary search for the first occurrence of key k takes O(log n) time.

# Binary search for the last occurrence of key k also takes O(log n) time.

# Extracting the elements with key k from the sorted table takes O(s) time, where
#  s is the number of items with key k.

# Overall, the time complexity of the find_all(k) operation is O(n * log n) 
# (sorting) + O(log n) (first occurrence) + O(log n) (last occurrence) + O(s)
#  (extraction) = O(n * log n) + O(log n) + O(log n) + O(s) = O(n * log n + s).

# So, the find_all(k) operation runs in O(s + log n) time, where n is the
# number of elements in the dictionary, and s is the number of items with 
# the given key k.

In [ ]:
# C-10.39 

# Although keys in a map are distinct, the binary search algorithm can be
# applied in a more general setting in which an array stores possibly duplicative
#  elements in nondecreasing order. Consider the goal of identifying the
# index of the leftmost element with key greater than or equal to given k.

# Does the find index method as given in Code Fragment 10.8 guarantee
# such a result? Does the find index method as given in Exercise R-10.21
# guarantee such a result? Justify your answers.

# code fragment 10.8
def _find_index(self, k, low, high):
    """
    Return index of the leftmost item with key greater than or equal to k.
    Return high + 1 if no such item qualifies.
    That is, j will be returned such that:
        all items of slice table[low:j] have key < k
        all items of slice table[j:high+1] have key >= k
    """
    if high < low:
        # no element qualifies
        return high + 1
    else:
        mid = (low + high) // 2
        if k == self._table[mid]._key:
            # found exact match
            return mid
        elif k < self._table[mid]._key:
            # note: may return mid
            return self._find_index(k, low, mid - 1)
        else:
            # answer is right of mid
            return self._find_index(k, mid + 1, high)

# R-10.21 is the recursive without the check for mid directly.

def _find_index(self, k, low, high): 
    """In contrast, the provided variant of _find_index recursively calls
     itself with a new range that includes both the upper and lower halves
      when the key at mid is less than k. This means that it might not
       return the index of the first occurrence of k but rather the index
    of the last occurrence of k."""
    if high < low:
        return high + 1
    else:
        mid = (low + high) // 2
        if self._table[mid]._key < k:
            return self._find_index(k, mid + 1, high)
        else:
            return self._find_index(k, low, mid -1)

# The find_index method as given in Code Fragment 10.8 does guarantee the result
#  of identifying the index of the leftmost element with a key greater than or
#  equal to the given key k in an array of distinct elements sorted in
#  nondecreasing order. This guarantee is achieved because it returns the
#  index where the element would be inserted to maintain the sorted order
#  (if it's not already in the array). Since the method returns the index 
# of the leftmost element greater than or equal to k, it satisfies the desired 
# outcome.

# On the other hand, the find_index method as given in Exercise R-10.21 does 
# not guarantee the result for finding the leftmost element greater than or 
# equal to k. The R-10.21 method returns the index of the rightmost occurrence 
# of an element equal to k in the array, or -1 if the element is not found.
#  This method is specifically designed for locating the rightmost occurrence 
# of an element with key k, not for identifying the leftmost element greater
#  than or equal to k. Therefore, it may not return the correct index for the
#  desired purpose.

# In summary:

# Code Fragment 10.8's find_index method guarantees the correct
#  result for finding the leftmost element greater than or equal to k.

# Exercise R-10.21's find_index method is not suitable for this specific
#  purpose, as it is intended for a different use case.


In [ ]:
# C-10.40 
# Suppose we are given two sorted search tables S and T, each with n entries
# (with S and T being implemented with arrays). Describe an O(log^2 n) time
#  algorithm for finding the kth smallest key in the union of the keys
# from S and T (assuming no duplicates).

# To find the kth smallest key in the union of two sorted arrays S and T, each with 
# n entries, you can use a modified binary search approach that achieves O(log^2 n)
#  time complexity. Here's a high-level description of the algorithm:
# 
# 1. Initialize two pointers, one for each array (S and T), and set them to point to 
# the first elements of their respective arrays.
# 
# 2. Use binary search to find the middle element (median) of each array. Let's call
#  them mid_S for array S and mid_T for array T.
# 
# 3. Compare mid_S and mid_T. If mid_S is less than mid_T, it means that the kth smallest 
# element cannot be in the first half of array S, or the second half of array T.
#  Conversely, if mid_S is greater than mid_T, it means that the kth smallest element
#  cannot be in the first half of array T, or the second half of array S.
# 
# 4. Based on the comparison result, you can eliminate approximately half of the 
# elements from one of the arrays. For example, if mid_S is smaller than mid_T, you
#  can discard the first half of array S and reduce the problem to finding the  
# (k - len(S) / 2)-th smallest element in the remaining part of array S and the
#  entire array T. If mid_S is greater, you discard the appropriate elements in
#  array T instead.
# 
# 5. Recursively repeat steps 2-4 with the reduced subproblems until you find the
#  kth smallest element.

# Here is a Python Like Pseudo code

def kth_smallest_in_sorted_arrays(S, T, k):
    if len(S) == 0:
        return T[k]
    if len(T) == 0:
        return S[k]
    
    mid_S = S[len(S) // 2]
    mid_T = T[len(T) // 2]
    
    if mid_S < mid_T:
        if k <= len(S) // 2 + len(T) // 2:
            return kth_smallest_in_sorted_arrays(S, T[:len(T) // 2], k)
        else:
            return kth_smallest_in_sorted_arrays(S[len(S) // 2 + 1:], T, k - len(T) // 2 - 1)
    else:
        if k <= len(S) // 2 + len(T) // 2:
            return kth_smallest_in_sorted_arrays(S[:len(S) // 2], T, k)
        else:
            return kth_smallest_in_sorted_arrays(S, T[len(T) // 2 + 1:], k - len(S) // 2 - 1)

# The algorithm divides the problem into smaller subproblems and discards approximately 
# half of the elements in each step, resulting in a time complexity of O(log^2 n), where
#  n is the size of the input arrays S and T.

In [ ]:
# TODO: this needs work

# C-10.41 Give an O(log n)-time solution for the previous problem.

# To find the kth smallest key in the union of two sorted arrays S and T, each 
# with n entries, in O(log n) time, you can use a modified binary search approach.
#  Here's the algorithm:

# 1. Initialize two pointers, left_S and left_T, to the beginning of arrays S and
#  T, respectively, and set k to the desired rank of the element.
# 
# 2. Perform a binary search on the range [1, n] to find the kth smallest element.
#  In each step of the binary search, do the following:
# 
#   a. Calculate the mid-rank as mid = (left_S + left_T) // 2.
# 
#   b. Calculate the mid-elements mid_S and mid_T corresponding to the mid-rank in
#  arrays S and T, respectively.
# 
#   c. If mid_S is less than mid_T, it means the kth smallest element is in the 
# range of elements before mid_S in array S and before mid_T in array T. Adjust
#  left_S accordingly: left_S = mid + 1.
# 
#   d. If mid_T is less than or equal to mid_S, it means the kth smallest element
#  is in the range of elements before mid_T in array T and before mid_S in array S.
#  Adjust left_T accordingly: left_T = mid + 1.
# 
# 3. Continue the binary search until you find the kth smallest element. The final
#  value of mid_S or mid_T will be the kth smallest element in the union of arrays
#  S and T.

def kthSmallestInSortedArrays(S, T, k):
    n = len(S)
    low, high = 0, n - 1
    
    while low <= high:
        mid = (low + high) // 2
        count = mid + 1  # Count of elements in the current search range
        
        if S[mid] < T[mid]:
            low = mid + 1
        else:
            high = mid - 1
        
        if count >= k:
            return min(S[mid], T[mid])
    
    # If k exceeds the total number of elements in both arrays, return -1 or handle it as needed.
    return -1  # Indicates an invalid k value


# This algorithm performs a binary search on the rank of the kth smallest element, reducing
#  the search range by half in each step. Therefore, it runs in O(log n) time, where
#  n is the size of the input arrays S and T.


S = [2, 4, 7, 9, 11]
T = [1, 3, 6, 8, 10]
k = 5

result = kthSmallestInSortedArrays(S, T, k)
print(result)  # Expected output: 6


In [3]:
# C-10.42 

# Suppose that each row of an n×n array A consists of 1’s and 0’s such that,
# in any row of A, all the 1’s come before any 0’s in that row. Assuming A
# is already in memory, describe a method running in O(n log n) time (not
# O(n^2) time!) for counting the number of 1’s in A.

# [1,1,1,1,1,0]
# [1,1,1,1,0,0]
# [1,1,1,1,0,0]
# [1,1,1,0,0,0]
# [1,1,1,0,0,0]
# [1,1,0,0,0,0]

# i think the solution is about skip lists

# You can indeed solve this problem using a modified binary search technique, which
#  is somewhat similar to how skip lists work. Here's an algorithm to count the
#  number of 1's in the given n×n array A in O(n log n) time:

# Start with the top-right element of the array A, which is A[0][n-1]. Initialize
#  two variables, row and col, to 0 and n-1, respectively. These variables 
# represent the current row and column you are checking.

# Initialize a variable count to 0, which will be used to keep track of the count of 1's.

# Repeat the following steps until row becomes n or col becomes -1 (i.e., until 
# you have checked all elements in the array):

# a. If A[row][col] is 1, it means there are 1's to the left of this element, so
#  increment count by col + 1 (the number of 1's in the current row) and move
#  to the next row by incrementing row by 1.

# b. If A[row][col] is 0, it means you need to move to the previous column by
#  decrementing col by 1.

# After the loop, count will contain the total count of 1's in the array A.

# The key idea here is that by starting at the top-right element and making
#  decisions based on the values encountered, you can efficiently traverse the
#  array while counting the 1's. Since you visit each element in the array
#  once, the algorithm runs in O(n log n) time.

def count_ones_in_sorted_array(A):
    n = len(A)
    row, col = 0, n - 1
    count = 0

    while row < n and col >= 0:
        if A[row][col] == 1:
            count += col + 1  # Increment count by the number of 1's in the current row
            row += 1  # Move to the next row
        else:
            col -= 1  # Move to the previous column
    
    return count

# This algorithm efficiently counts the 1's in the given array while avoiding unnecessary traversals
#  of 0's, resulting in a time complexity of O(n log n).

# Example 2D array (6x6) with sorted rows
array_A = [
    [1, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 0, 0],
    [1, 1, 1, 1, 0, 0],
    [1, 1, 1, 0, 0, 0],
    [1, 1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0, 0]
]

# Count the number of 1's in the array
count = count_ones_in_sorted_array(array_A)

# Expected result: There are 21 occurrences of 1 in the array.
print("Number of 1's in the array:", count)


Number of 1's in the array: 21


In [13]:
"""Genius"""

# C-10.43 
# Given a collection C of n cost-performance pairs (c, p), describe an 
# algorithm for finding the maxima pairs of C in O(nlog n) time.

# To find the maximum pairs (c, p) from a collection C of n cost-performance 
# pairs in O(n log n) time, you can use the following algorithm, which is 
# based on sorting:

# Make a list of pairs (c, p) from the collection C.

# Sort the list of pairs in non-decreasing order of performance (p). This can
#  be done using a stable sorting algorithm, such as mergesort or Timsort, which
#  has a time complexity of O(n log n).

# Initialize a variable max_c to the minimum possible cost (e.g., negative infinity).

# Initialize an empty list max_pairs to store the maximum pairs.

# Iterate through the sorted list of pairs from highest performance (end of the list) to
#  lowest performance (beginning of the list):

# a. For each pair (c, p), if c is greater than max_c, add (c, p) to max_pairs and 
# update max_c to c.

# The max_pairs list will now contain the maximum pairs of (c, p) based on their 
# performance, and max_c will contain the maximum cost among those pairs.

# This algorithm guarantees that you find the maximum pairs based on performance
#  while keeping track of the maximum cost. The sorting step dominates the time
#  complexity, resulting in O(n log n) time complexity for the entire algorithm.

# this is cool, but it doesnt work.
def find_maximum_pairs_(computer_parts):
    # Sort the list of pairs based on performance (ascending order)
    sorted_parts = sorted(computer_parts, key=lambda x: x[1])

    max_cost = float('-inf')
    max_pairs = []

    # Iterate through sorted pairs
    for cost, performance in sorted_parts:
        if cost > max_cost:
            max_pairs.append((cost, performance))
            max_cost = cost

    return max_pairs
    
def find_maximum_pairs(computer_parts):
    # Make a dictionary to store the maximum performance for each cost
    max_performance_dict = {}

    # Iterate through the computer parts
    for cost, performance in computer_parts:
        if cost not in max_performance_dict or performance > max_performance_dict[cost]:
            max_performance_dict[cost] = performance

    # Make a list of maximum pairs based on the dictionary
    max_pairs = [(cost, perf) for cost, perf in max_performance_dict.items()]
    # Sort the maximum pairs based on cost in ascending order
    max_pairs.sort(key=lambda x: x[0])

    return max_pairs

# Example usage:
computer_parts = [
    (100, 50),
    (200, 60),
    (150, 55),
    (150, 58),
    (150, 59),
    (250, 70),
    (180, 58)
]

max_pairs = find_maximum_pairs(computer_parts)

print("Maximum Pairs (Cost, Performance):")
for cost, performance in max_pairs:
    print(f"Cost: {cost}, Performance: {performance}")

Maximum Pairs (Cost, Performance):
Cost: 100, Performance: 50
Cost: 150, Performance: 59
Cost: 180, Performance: 58
Cost: 200, Performance: 60
Cost: 250, Performance: 70


In [ ]:
# C-10.44 

# Show that the methods above(p) and prev(p) are not actually needed to
# efficiently implement a map using a skip list. 
# 
# That is, we can implement  insertions and deletions in a skip list using a 
# strictly top-down, scan-forward approach, without ever using the above or prev methods. (Hint:
# In the insertion algorithm, first repeatedly flip the coin to determine the
# level where you should start inserting the new entry.

# The above(p) and prev(p) methods in a skip list are used to efficiently navigate and search
#  the skip list. While they can be helpful, they are not strictly necessary to implement 
# insertions and deletions in a skip list. You can indeed implement these operations using
#  a top-down, scan-forward approach without explicitly using above(p) or prev(p) methods. 
# Here's a high-level overview of how you can achieve this:

# Insertion in a Skip List:
# 
# 1. Start from the top-level of the skip list.

# 2. Flip a coin to determine the level where you should start inserting the new entry. 

# 3. This is essentially simulating the use of above(p) without directly calling it.

# 4. Traverse the skip list from the top to the bottom while keeping track of the nodes 
# visited at each level.

# 5. At each level, check if the next node's key is greater than the key of the element
#  you want to insert. If it is, insert the new element before that node. If not, continue
#  moving forward.
# When you reach the bottom level, insert the new element into the bottom-level list.

# Deletion in a Skip List:

# 1. Start from the top-level of the skip list.

# 2. Traverse the skip list from the top to the bottom while keeping track of the nodes
#  visited at each level.
# 3. At each level, check if the next node's key matches the key of the element you want 
# to delete. If it does, remove the reference to that node. If not, continue moving forward.

# 4. When you reach the bottom level, you have deleted the element from the skip list.

# By following this approach, you can insert and delete elements in a skip list without
#  explicitly using above(p) or prev(p) methods. The coin-flipping mechanism and the
#  top-down scan-forward traversal allow you to efficiently navigate and modify the 
# skip list structure.

# These methods ensure that you maintain the skip list's structure and properties 
# while performing insertions and deletions efficiently.

In [ ]:
# C-10.45 
# Describe how to modify a skip-list representation so that index-based
# operations, such as retrieving the item at index j, can be performed in
# O(log n) expected time.

# To modify a skip-list representation to support index-based operations 
# in O(log n) expected time, you can use a variant known as an "indexed skip
#  list" or "indexed skip list with levels." 

# Here's how you can achieve this:

# 1. Augment Each Node with Size Information:

#   - Each node in the skip list should store additional information about the 
# size of the sublist below it. This information represents the number of 
# nodes in the sublist that can be reached by following the forward pointers.

# 2. Build a Forward Index:

#   - Maintain a separate index that stores pointers to nodes at regular intervals. 
# These intervals should be determined based on the total size of the skip list.

#   - For example, you can maintain an index for every 1/4th or 1/8th of the total size.

# 3. Perform Index-Based Operations:

#   - To retrieve the item at index j, start at the top-left corner of the skip list (the head).

#   - Initialize an index variable to zero.

#   - Traverse the skip list from the top level to the bottom level, always moving forward 
# to the next node.

#  -  As you move, check the size information of the current node. If adding its size to the
#  index variable keeps you below or equal to j, move to that node and increment the index
#  variable by the node's size.

#  -  Continue this process until you reach the bottom level.

#   - Once you reach the bottom level, you'll have found the node that corresponds 
# to the item at index j.

# 4. Efficiency Analysis:

#   - With this modification, index-based operations, such as retrieving the item at
#  index j, can be performed in O(log n) expected time.

#   - The use of the forward index allows you to skip larger portions of the skip
#  list, reducing the number of nodes you need to traverse.

# By maintaining size information and a forward index, you can efficiently support
#  index-based operations while still benefiting from the skip list's overall
#  structure, which ensures logarithmic expected time complexity for 
# searching, insertion, and deletion operations.

# This modification strikes a balance between maintaining the skip list's inherent
#  properties and enabling efficient index-based operations.



In [6]:
# C-10.46 
# For sets S and T, the syntax S ˆ T returns a new set that is the 
# symmetric difference, that is, a set of elements that are in precisely one of S or T. 
# This syntax is supported by the special xor method. Provide an
# implementation of that method in the context of the MutableSet abstract
# base class, relying only on the five primary abstract methods of that class.

from collections.abc import MutableSet

class MySet(MutableSet):
    def __init__(self, iterable=None):
        self.elements = set()
        if iterable is not None:
            # update method for sets. 
            self |= set(iterable)

    def __contains__(self, value):
        return value in self.elements

    def __iter__(self):
        return iter(self.elements)

    def __len__(self):
        return len(self.elements)

    def add(self, value):
        self.elements.add(value)

    def discard(self, value):
        self.elements.discard(value)

    def __xor__(self, other):
        # Make a new set to store the symmetric difference
        result = MySet(self)

        # Remove elements that are in both sets (intersection)
        for element in other:
            if element in result:
                result.discard(element)
            else:
                result.add(element)

        # Remove elements that are in the original set
        for element in self:
            if element in other:
                result.discard(element)
            else:
                result.add(element)

        return result

    def __str__(self) -> str:
        return f"A set object with elements {[elem for elem in iter(self)]}"

# Example usage:
set1 = MySet([1, 2, 3, 4, 5])
set2 = MySet([3, 4, 5, 6, 7])

result = set1 ^ set2  # Calculate the symmetric difference
print(result)  # Output: {1, 2, 6, 7}


A set object with elements [1, 2, 6, 7]


In [7]:
# C-10.47 

# In the context of the MutableSet abstract base class, describe a concrete
# implementation of the and method, which supports the syntax S & T 
# for computing the intersection of two existing sets

from collections.abc import MutableSet

class MySet(MutableSet):
    def __init__(self, iterable=None):
        self.elements = set()
        if iterable is not None:
            self |= set(iterable)

    def __contains__(self, value):
        return value in self.elements

    def __iter__(self):
        return iter(self.elements)

    def __len__(self):
        return len(self.elements)

    def add(self, value):
        self.elements.add(value)

    def discard(self, value):
        self.elements.discard(value)

    def __and__(self, other_set):
        # Make a new set to store the intersection
        intersection = MySet()
        
        # Iterate through the elements in self and check if they are in other_set
        for elem in self.elements:
            if elem in other_set:
                intersection.add(elem)
        
        return intersection

    def __str__(self):
        return str(self.elements)

set1 = MySet([1, 2, 3, 4, 5])
set2 = MySet([3, 4, 5, 6, 7])

intersection_set = set1 & set2
print(intersection_set)  # Output: {3, 4, 5}


{3, 4, 5}


In [ ]:
# C-10.48 

# An inverted file is a critical data structure for implementing a search engine
#  or the index of a book. Given a document D, which can be viewed
# as an unordered, numbered list of words, an inverted file is an ordered list
# of words, L, such that, for each word w in L, we store the indices of the
# places in D where w appears. Design an efficient algorithm for constructing L from D.

# To construct an inverted file, you can use a combination of data structures to efficiently 
# Make an ordered list of words, L, along with the indices of places in D where each word
#  appears. Here's an algorithm to achieve this:
# 
# 1) Initialize an empty dictionary inverted_file, where the keys are words, and the values
#  are sets to store the indices of places where each word appears. Initialize an empty list
#  word_list to store the ordered list of words.
# 
# 2) Split the document D into words. You can do this by splitting the document using spaces
#  or any other delimiter, and removing punctuation marks.
# 
# 3) Iterate through the words in D along with their indices. Keep track of the current index
#  as you iterate.
# 
# 4) For each word encountered at index i, check if it exists in the inverted_file dictionary.
#  If it doesn't exist, Make a new entry with the word as the key and initialize an empty
#  set as the value. Append the word to the word_list.
#  
# 5) Add the current index i to the set associated with the word in the inverted_file dictionary.
# 
# 6) Repeat steps 4 and 5 for all words in the document.
# 
# 7) After processing all words in the document, sort the word_list in lexicographic 
# (or other desired) order.
# 
# 8) You now have an ordered list of words L, and for each word, you have a set of indices
#  indicating where it appears in the document. This forms the inverted file.

def construct_inverted_file(document):
    inverted_file = {}
    word_list = []
    
    # Split the document into words and process them along with their indices
    words = document.split()  # You can modify the splitting logic as needed
    
    for i, word in enumerate(words):
        # Remove punctuation marks (you may need more sophisticated processing)
        word = word.strip('.,!?()[]{}":;')
        
        if word not in inverted_file:
            inverted_file[word] = set()
            word_list.append(word)
        
        inverted_file[word].add(i)
    
    # Sort the word_list
    word_list.sort()
    
    return word_list, inverted_file

# Example usage:
document = "This is a sample document. It contains words. Words appear in this document."
word_list, inverted_file = construct_inverted_file(document)

# Print the ordered list of words and the inverted file
print("Ordered List of Words (L):", word_list)
print("\nInverted File:")
for word in word_list:
    print(f"{word}: {inverted_file[word]}")

# This algorithm efficiently constructs an ordered list of words and the associated inverted 
# file with the indices where each word appears in the document. The resulting data 
# structures can be used for various text retrieval tasks, such as search engines
#  or book indexing.

In [ ]:
# C-10.49 

# Python’s collections module provides an OrderedDict class that is unrelated
#  to our sorted map abstraction. An OrderedDict is a subclass of the
# standard hash-based dict class that retains the expected O(1) performance
# for the primary map operations, but that also guarantees that the iter
# method reports items of the map according to first-in, first-out (FIFO)
# order. That is, the key that has been in the dictionary the longest is reported
#  first. (The order is unaffected when the value for an existing key
# is overwritten.) Describe an algorithmic approach for achieving such performance.

# To achieve the behavior of an OrderedDict in Python, which guarantees first-in,
#  first-out (FIFO) order for iterating through items, you can implement a custom
#  data structure that combines the functionality of a hash table and a doubly
#  linked list. The linked list will keep track of the order of insertion for items.
# 
# Here's an algorithmic approach to implement this custom OrderedDict-like data structure:
# 
# Make a class, let's call it OrderedDictCustom, which will have the following attributes:
# 
# A hash table (dictionary) to store key-value pairs.
# A doubly linked list to maintain the order of insertion.
# Define a node structure for the doubly linked list. Each node should store the
#  key, value, and references to the previous and next nodes.
# 
# Implement the following methods for OrderedDictCustom:
# 
# __init__: Initialize the hash table and Make a dummy head and tail for the doubly linked list.

# __getitem__: Retrieve the value for a given key from the hash table.

# __setitem__: Add or update a key-value pair in the hash table. Also, update the linked list
#  to maintain the order of insertion.

# __delitem__: Remove a key-value pair from the hash table. Update the linked list
#  to remove the corresponding node.

# __iter__: Iterate through the linked list in FIFO order, starting from the
#  dummy head node.

# __len__: Return the number of key-value pairs in the hash table.

# Optionally, add other methods as needed, such as keys, values, etc., to match the
#  behavior of Python's dict.

# When adding a new key-value pair to the dictionary using __setitem__, Make a 
# new node and insert it at the end of the linked list (before the dummy tail node).
#  This ensures that the most recently added item is at the end of the list, and
#  the oldest item is at the beginning.
# 
# When removing a key-value pair using __delitem__, find the corresponding node
#  in the linked list and remove it. Update the references of the previous and
#  next nodes accordingly.
# 
# When iterating through the items using __iter__, start from the dummy head
#  node and move to the next node in the list until you reach the dummy tail
#  node. This will give you items in FIFO order.

class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class OrderedDictCustom:
    def __init__(self):
        self.hash_table = {}
        self.head = Node(None, None)  # Dummy head
        self.tail = Node(None, None)  # Dummy tail
        self.head.next = self.tail
        self.tail.prev = self.head

    def __getitem__(self, key):
        if key in self.hash_table:
            node = self.hash_table[key]
            return node.value
        raise KeyError(f"Key not found: {key}")

    def __setitem__(self, key, value):
        if key in self.hash_table:
            # Update existing key
            node = self.hash_table[key]
            node.value = value
        else:
            # Insert a new key
            node = Node(key, value)
            self.hash_table[key] = node
            # Insert the new node at the end of the linked list
            prev_tail = self.tail.prev
            prev_tail.next = node
            node.prev = prev_tail
            node.next = self.tail
            self.tail.prev = node

    def __delitem__(self, key):
        if key in self.hash_table:
            node = self.hash_table[key]
            del self.hash_table[key]
            # Remove the node from the linked list
            prev_node = node.prev
            next_node = node.next
            prev_node.next = next_node
            next_node.prev = prev_node
        else:
            raise KeyError(f"Key not found: {key}")

    def __iter__(self):
        current = self.head.next
        while current != self.tail:
            yield current.key
            current = current.next

    def __len__(self):
        return len(self.hash_table)

# Example usage:
ordered_dict = OrderedDictCustom()
ordered_dict["apple"] = 1
ordered_dict["banana"] = 2
ordered_dict["cherry"] = 3

for key in ordered_dict:
    print(key, ordered_dict[key])  # Outputs in FIFO order: apple, banana, cherry

# This custom OrderedDict-like data structure maintains the order of insertion
#  while providing O(1) time complexity for basic operations.    


In [ ]:
"""Not right now."""

# P-10.50 

# Perform a comparative analysis that studies the collision rates for various
# hash codes for character strings, such as various polynomial hash codes
# for different values of the parameter a. Use a hash table to determine
# collisions, but only count collisions where different strings map to the
# same hash code (not if they map to the same location in this hash table).
# Test these hash codes on text files found on the Internet

In [ ]:
"""Not right now."""

# P-10.51 
# Perform a comparative analysis as in the previous exercise, but for 10-digit
# telephone numbers instead of character strings

In [7]:
# P-10.52 
# Implement an OrderedDict class, as described in Exercise C-10.49, 
# ensuring that the primary map operations run in O(1) expected time.

# To implement an OrderedDict class that guarantees O(1) expected time for
#  primary map operations, you can use a combination of a hash table and a
#  doubly linked list. This approach ensures that key-value pairs are 
# stored in the order they were added and allows for efficient
#  insertion, deletion, and retrieval.

class ListNode:
    def __init__(self, key=None, value=None):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class OrderedDict:
    def __init__(self):
        self.hash_map = {}
        self.head = ListNode()
        self.tail = ListNode()
        self.head.next = self.tail
        self.tail.prev = self.head

    def _move_to_end(self, node):
        # Move the given node to the end of the linked list
        prev_node = node.prev
        next_node = node.next

        prev_node.next = next_node
        next_node.prev = prev_node

        node.prev = self.tail.prev
        node.next = self.tail

        self.tail.prev.next = node
        self.tail.prev = node

    def __getitem__(self, key):
        if key not in self.hash_map:
            raise KeyError(key)

        node = self.hash_map[key]
        self._move_to_end(node)
        return node.value

    def __setitem__(self, key, value):
        if key in self.hash_map:
            node = self.hash_map[key]
            node.value = value
            self._move_to_end(node)
        else:
            new_node = ListNode(key, value)
            self.hash_map[key] = new_node
            new_node.prev = self.tail.prev
            new_node.next = self.tail
            self.tail.prev.next = new_node
            self.tail.prev = new_node

    def __delitem__(self, key):
        if key not in self.hash_map:
            raise KeyError(key)

        node = self.hash_map[key]
        del self.hash_map[key]
        node.prev.next = node.next
        node.next.prev = node.prev

    def __iter__(self):
        current = self.head.next
        while current != self.tail:
            yield current.key
            current = current.next

    def items(self):
        return ((key, self[key]) for key in self)

    def __len__(self):
        return len(self.hash_map)

    def __contains__(self, key):
        return key in self.hash_map

    def move_to_end(self, key):
        if key not in self.hash_map:
            raise KeyError(key)

        node = self.hash_map[key]
        self._move_to_end(node)

    def popitem(self, last=True):
        if not self:
            raise KeyError("OrderedDict is empty")
        key = self.tail.prev.key if last else self.head.next.key
        del self[key]
        return (key, self[key])

    def pop(self, key, default=None):
        if key in self.hash_map:
            value = self[key]
            del self[key]
            return value
        return default

    def clear(self):
        self.hash_map.clear()
        self.head.next = self.tail
        self.tail.prev = self.head

    def keys(self):
        return iter(self)

    def values(self):
        return (self[key] for key in self)

    def update(self, *args, **kwargs):
        if args:
            if len(args) > 1:
                raise TypeError("update expected at most 1 argument, got %d" % len(args))
            other = args[0]
            if hasattr(other, "keys"):
                for key in other.keys():
                    self[key] = other[key]
            else:
                for key, value in other:
                    self[key] = value
        for key in kwargs:
            self[key] = kwargs[key]

    def setdefault(self, key, default=None):
        if key not in self:
            self[key] = default
        return self[key]

    def __eq__(self, other):
        if isinstance(other, OrderedDict):
            return len(self) == len(other) and all(k1 == k2 and v1 == v2 for (k1, v1), (k2, v2) in zip(self.items(), other.items()))
        return False

    def __ne__(self, other):
        return not self == other

    def __repr__(self):
        if not self:
            return "%s()" % (self.__class__.__name__,)
        return "%s(%r)" % (self.__class__.__name__, self.items())

    def __str__(self):
        return repr(self)

# Example usage:
od = OrderedDict()
od['a'] = 1
od['b'] = 2
od['c'] = 3

print([elem for elem in od.keys()])  # Output: ['a', 'b', 'c']
print([next(od.values()) for _ in range(3)])   # Output: [1, 2, 3]

['a', 'b', 'c']
[1, 2, 3]


In [ ]:
# P-10.53 
# Design a Python class that implements the skip-list data structure. Use
# this class to Make a complete implementation of the sorted map ADT.

# Implementing a skip-list data structure from scratch is a complex task, and
#  providing a complete implementation of the sorted map ADT using skip-lists
#  in a single response is impractical. Instead, I'll provide a high-level
#  overview of how you can design a Python class for a skip-list and suggest
#  the steps to adapt it for use as a sorted map ADT.

import random

class SkipListNode:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.forward = []

class SkipList:
    def __init__(self):
        self.head = SkipListNode(float('-inf'), None)
        self.tail = SkipListNode(float('inf'), None)
        self.head.forward = [self.tail]
        self.max_level = 0

    def random_level(self):
        level = 1
        while random.random() < 0.5 and level <= self.max_level:
            level += 1
        return level

    def insert(self, key, value):
        update = [None] * (self.max_level + 1)
        node = self.head

        for level in range(self.max_level, -1, -1):
            while node.forward[level] and node.forward[level].key < key:
                node = node.forward[level]
            update[level] = node

        level = self.random_level()

        if level > self.max_level:
            for i in range(self.max_level + 1, level + 1):
                update.append(self.head)
            self.max_level = level

        new_node = SkipListNode(key, value)
        new_node.forward = [None] * (level + 1)

        for i in range(level + 1):
            new_node.forward[i] = update[i].forward[i]
            update[i].forward[i] = new_node

    def delete(self, key):
        update = [None] * (self.max_level + 1)
        node = self.head

        for level in range(self.max_level, -1, -1):
            while node.forward[level] and node.forward[level].key < key:
                node = node.forward[level]
            update[level] = node

        if node.forward[0] and node.forward[0].key == key:
            deleted_node = node.forward[0]
            for level in range(len(deleted_node.forward)):
                update[level].forward[level] = deleted_node.forward[level]
            del deleted_node

    def search(self, key):
        node = self.head

        for level in range(self.max_level, -1, -1):
            while node.forward[level] and node.forward[level].key < key:
                node = node.forward[level]

        if node.forward[0] and node.forward[0].key == key:
            return node.forward[0].value
        else:
            return None


# This code defines a basic skip-list data structure with insertion, deletion, and
#  search operations. To adapt it for use as a sorted map ADT, you can store
#  key-value pairs in the skip-list nodes, use the insert method for adding
#  key-value pairs, and modify the search method to return values associated
#  with keys. You can also implement additional map-related methods like
#  __getitem__, __setitem__, __delitem__, and others to fully support the 
# sorted map ADT.

# Please note that this is a simplified implementation, and skip-lists can
#  become more complex as you add features and optimizations. A complete
#  sorted map implementation would also include error handling, balancing, and
# other features required for a practical data structure.

In [ ]:
"""A challange for later"""

# P-10.54 
# 
# Extend the previous project by providing a graphical animation of the
# skip-list operations. Visualize how entries move up the skip list during
# insertions and are linked out of the skip list during removals. Also, in a
# search operation, visualize the scan-forward and drop-down actions.

# Creating a graphical animation of skip-list operations is a more involved
#  project that goes beyond simple text-based programming. To achieve this, you
#  would need to use a graphical user interface (GUI) library like Tkinter, PyQt, or
#  Pygame, which allows you to Make visual elements and animations.
# 
# Here's a high-level overview of how you can approach this project using Python
#  and a GUI library:
# 
# Choose a GUI Library: Select a Python GUI library that suits your needs and skills. 
# Tkinter is the most straightforward choice for simple GUIs, while PyQt and Pygame
#  offer more advanced features and flexibility.
# 
# Make a Skip-List Visualization Class: Define a class that represents the skip-list 
# and handles its visualization. This class should include methods for 
# inserting, deleting, and searching for elements while updating the graphical
#  representation.
# 
# Visualize Insertions: When you insert an element into the skip-list, visually 
# represent the steps involved in choosing the level and updating the pointers. 
# Use animations to show how the new element moves up the levels.
# 
# Visualize Deletions: When you delete an element, visualize how it is unlinked 
# from the skip-list structure. Use animations to demonstrate how the pointers 
# are adjusted.
# 
# Visualize Searches: During a search operation, show how the algorithm scans 
# forward and drops down the levels as it searches for the target element. Highlight
#  the visited nodes and display the path taken.
# 
# Implement User Controls: Add buttons, sliders, or other user controls to interact
#  with the skip-list, such as inserting, deleting, and searching for elements.
# 
# Make an Animated GUI: Build the graphical user interface that displays the skip-list
#  and responds to user input. Update the visualization in real-time as operations 
# are performed.
# 
# Testing and Debugging: Test your graphical skip-list with various input scenarios
#  to ensure that it behaves as expected and that the animations are accurate.
# 
# Optimize and Refine: Optimize the animation speed, add labels and tooltips, and 
# refine the user interface to make it user-friendly and informative.
# 
# Documentation and Presentation: Provide clear documentation for your project, explaining
#  how to use it and how the animations work. Consider creating a presentation or demo
#  to showcase your project's features.
# 
# This project will require a solid understanding of both skip-list data structures and 
# GUI programming with the chosen library. It's a challenging but rewarding endeavor that
#  combines data structure implementation with visualization and interaction. Depending 
# on your experience and goals, it may take some time to complete, but it can be a 
# valuable learning experience.

In [8]:
"""A challange for later"""

# P-10.55 

# Write a spell-checker class that stores a lexicon of words, W, in a Python
# set, and implements a method, check(s), which performs a spell check
# on the string s with respect to the set of words, W. 
# 
# If s is in W, then  the call to check(s) returns a list containing only s, as it is assumed to
# be spelled correctly in this case. 
# 
# If s is not in W, then the call to check(s) returns a list of every word in
#  W that might be a correct spelling of s. 
# 
# Your program should be able to handle all the common ways that s might be a
# misspelling of a word in W, including swapping adjacent characters in a
# word, inserting a single character in between two adjacent characters in a
# word, deleting a single character from a word, and replacing a character in
# a word with another character. For an extra challenge, consider phonetic
# substitutions as well.

# here is a simplified approach as a guide.

# Implementing a spell-checker class that handles various types of misspellings
#  is a complex task. Here's a Python class that provides basic functionality 
# for spell-checking using Levenshtein distance (edit distance) as a measure 
# for identifying possible correct spellings. The class uses a set of words as
#  the lexicon.

import numpy as np

class SpellChecker:
    def __init__(self, lexicon):
        self.lexicon = set(lexicon)

    def check(self, s):
        if s in self.lexicon:
            return [s]  # The word is correct, return it

        # Generate possible corrections
        candidates = self._generate_candidates(s)

        # Calculate edit distances
        distances = {word: self._edit_distance(s, word) for word in candidates}

        # Sort candidates by edit distance (ascending)
        sorted_candidates = sorted(candidates, key=lambda word: distances[word])

        return sorted_candidates

    def _generate_candidates(self, s):
        # Generate possible corrections by swapping, inserting, deleting, and replacing characters
        candidates = set()

        # Swapping adjacent characters
        for i in range(len(s) - 1):
            swapped = s[:i] + s[i + 1] + s[i] + s[i + 2:]
            candidates.add(swapped)

        # Inserting a character
        for i in range(len(s) + 1):
            for c in 'abcdefghijklmnopqrstuvwxyz':
                inserted = s[:i] + c + s[i:]
                candidates.add(inserted)

        # Deleting a character
        for i in range(len(s)):
            deleted = s[:i] + s[i + 1:]
            candidates.add(deleted)

        # Replacing a character
        for i in range(len(s)):
            for c in 'abcdefghijklmnopqrstuvwxyz':
                replaced = s[:i] + c + s[i + 1:]
                candidates.add(replaced)

        return candidates

    def _edit_distance(self, word1, word2):
        # Calculate Levenshtein distance between two words
        m, n = len(word1), len(word2)
        dp = np.zeros((m + 1, n + 1))

        for i in range(m + 1):
            dp[i][0] = i

        for j in range(n + 1):
            dp[0][j] = j

        for i in range(1, m + 1):
            for j in range(1, n + 1):
                cost = 0 if word1[i - 1] == word2[j - 1] else 1
                dp[i][j] = min(dp[i - 1][j] + 1, dp[i][j - 1] + 1, dp[i - 1][j - 1] + cost)

        return dp[m][n]

# Example usage:
lexicon = ["apple", "banana", "cherry", "grape", "orange"]
spell_checker = SpellChecker(lexicon)
word = "aple"
corrections = spell_checker.check(word)
print(f"Possible corrections for '{word}': {corrections}")


Possible corrections for 'aple': ['aple', 'asple', 'apled', 'caple', 'axple', 'kple', 'pple', 'apae', 'aplb', 'aplu', 'alle', 'apve', 'faple', 'avple', 'apmle', 'apleo', 'acle', 'aplve', 'aule', 'ople', 'apcle', 'aplc', 'aplv', 'ale', 'apleh', 'iple', 'aplue', 'aplo', 'uple', 'taple', 'apbe', 'apze', 'ayle', 'awle', 'aplm', 'ahple', 'aplec', 'qaple', 'aplke', 'ample', 'aplje', 'ahle', 'tple', 'nple', 'aiple', 'appe', 'apsle', 'apble', 'azle', 'aplx', 'waple', 'eaple', 'apde', 'apqe', 'adple', 'apse', 'apdle', 'apule', 'anple', 'ape', 'apole', 'apnle', 'apvle', 'aplj', 'aplhe', 'dple', 'akple', 'aplxe', 'aphle', 'uaple', 'apfle', 'aplez', 'anle', 'apre', 'atple', 'aplte', 'aale', 'aphe', 'apje', 'laple', 'apee', 'abple', 'apele', 'aplef', 'axle', 'aplie', 'aplee', 'adle', 'aply', 'apxe', 'apll', 'akle', 'aplew', 'kaple', 'aploe', 'bple', 'afle', 'apleb', 'aptle', 'baple', 'agple', 'aplce', 'aplei', 'arle', 'atle', 'aplge', 'aplh', 'apjle', 'aplea', 'zple', 'apqle', 'apke', 'aplex', 'dap